# Proyecto Calidad de Datos Bancoomeva Cliente Persona Jurídica

# Configuración e importe de paquetes

se importan los paquetes de python necesarios

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
#from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import numpy as np

Configuración del controlador e inicio de sesion Spark

In [2]:
path_jar_driver = '/home/jovyan/code/java_sqlServer/mssql-jdbc-12.4.2.jre8.jar'

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver) \
    .set('spark.driver.memory', '8g')  # Ajusta el tamaño de la memoria del driver según sea necesario


spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession


/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Conexión a fuente de datos y acceso a los datos

### Detalles de la conexión a la base de datos SQL de Azure

In [4]:
# Detalles de conexión a la base de datos SQL de Azure
url = "jdbc:sqlserver://sqlserver-calidadgobierno.database.windows.net:1433;database=sqldb-calidadgobierno"
usuario = "sqladmin@sqlserver-calidadgobierno"
contraseña = "y.HCuP4gJBb?)MS/6~W!+;"
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

### Función para convertir un Dataframe en una tabla para mi base de datos 

In [5]:
from pyspark.sql import DataFrame
def cargar_dataframe_a_tabla(dataframe: DataFrame, nombre_tabla: str):
    """
    Método para cargar un DataFrame en una tabla de una base de datos.

    Parámetros:
    - dataframe: DataFrame que se cargará en la tabla.
    - nombre_tabla: Nombre de la tabla en la base de datos.
    """
    # Escribe el DataFrame en la base de datos como una tabla
    dataframe.write.jdbc(url=url, table=nombre_tabla, mode="overwrite", properties={"user": usuario, "password": contraseña, "driver": driver})

In [6]:
from pyspark.sql import SparkSession

def leer_tabla_a_dataframe(url, tabla, usuario, contraseña, driver):

    # Configurar la conexión a la base de datos
    properties = {
        "user": usuario,
        "password": contraseña,
        "driver": driver
    }

    # Leer la tabla y convertirla en un DataFrame
    df = spark.read.jdbc(url=url, table=tabla, properties=properties)

    return df


# Nombre de la tabla que se desea leer
nombre_tabla = "Calidad.clienteJuridico2"

# Llamar a la función para leer la tabla y obtener el DataFrame
df_spark = leer_tabla_a_dataframe(url, nombre_tabla, usuario, contraseña, driver)

# Mostrar el DataFrame con el campo TEL_REP convertido a float
df_spark.show()


# Seleccionar los primeros 500 registros y sobrescribir el DataFrame original
#df_spark = df_spark.limit(40)

# Obtener el número de filas
num_filas = df_spark.count()

# Obtener el número de columnas
num_columnas = len(df_spark.columns)

print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

# Mostrar el esquema del DataFrame obtenido
df_spark.printSchema()

+------+------+---------+--------------------+------+--------------------+----------+--------+--------------------+-----------+-----------+----------+-----------------+------------------------+------------------+-------------------+----------+---------------------+----------------------------+----------------------------+--------------------+---------------------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------------------------------------+------------------+-----------------+----+--------+---------------+-------+--------------------+--------------------+----------------+----------+--------------------+-------------------------------+-------------+---------------------+--------------------+----------------+--------------+----------+--------+---------+-------+---------+----------------------+--------------------+-------------+----------+--------------------+--------------------+-------------------+-------------+----------------

In [ ]:
#df_spark = df_spark.filter(col("FechaCorte") == "29/02/2024")

# Mostrar el DataFrame filtrado
#df_spark.show()

#num_filas = df_spark.count()

# Obtener el número de columnas
#num_columnas = len(df_spark.columns)

#print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

# Extracción del Dataframe desde un archivo plano xlsb

In [ ]:
#pip install pyxlsb
# Leer el archivo CSV y crear un DataFrame
#df_spark = spark.read.csv("/home/jovyan/code/Bancoomeva_Oficial/BD_ClientesPN_v001_20240131_final.csv", header=True, sep=";" , inferSchema=True)

# Mostrar el DataFrame
#df_spark.show()

# Obtener el número de filas/columnas
#num_filas = df_spark.count()
#num_columnas = len(df_spark.columns)

#print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

In [ ]:
#cargar_dataframe_a_tabla(df_spark,"Calidad.ClienteOrigen2")

# Definición de Reglas/Filtros para la Calidad de los datos: 

## Campo REP_LEGAL

Regla 1: Primer nombre y primer apellido deben ser obligatorios. Segundo nombre y segundo apellido pueden ser nulos.

In [ ]:
from pyspark.sql.functions import col, size, split

REP_LEGAL_Regla_PrimerNombre_PrimerApellido = size(split(col('REP_LEGAL'), ' ')) < 2

Regla 2: Campo no debe contener números (el dato contenga caracteres alfabeticos)

In [ ]:
REP_LEGAL_Regla_NoNumeros = col('REP_LEGAL').rlike(r'\d')

Regla 3: Campo no debe contener caracteres especiales (a excepción de la Ñ)

In [ ]:
REP_LEGAL_Regla_NoCaracteresEspeciales = col('REP_LEGAL').rlike(r'[^a-zA-ZñÑ\s]')

Regla 4: Validación de espacios entre segmento de caracteres(palabras)

In [ ]:
REP_LEGAL_Regla_AlMenosUnEspacio = (
    ~col('REP_LEGAL').rlike(r'\s')  # Verifica si hay al menos un espacio en el campo
)

Regla 5: Que no exista solo un punto (.), coma (,), asteriscos (*)

In [ ]:
# Regla para validar que el campo "NOMCLI" no contenga puntos, comas ni asteriscos
REP_LEGAL_Regla_NoPuntosComasAsteriscos = col('REP_LEGAL').rlike(r'[.,*]')

Regla 6: No debe contener sucesion del mismo carácter mas de 3 veces (XXXX, AAAAAA,etc.)

In [ ]:
# Regla para validar que el campo "NOMCLI" no tenga una secuencia de caracteres repetidos más de tres veces
REP_LEGAL_Regla_NoSecuenciaRepetida = col('REP_LEGAL').rlike(r'(.)\1{3,}')

Regla 7:  El campo no debe contener  una sola letra minúscula o mayúscula

In [ ]:
# Regla para validar que el campo "NOMCLI" no tenga una sola letra en mayúscula o minúscula
REP_LEGAL_Regla_NoUnaSolaLetra = ~col('REP_LEGAL').rlike(r'^(?![a-zA-Z]$).*$')

Regla 8: El campo no debe tener con silabas compuestas por 2 letras, es decir, dos letras combinadas o repetidas de cualquier forma en minúscula o mayúscula

In [ ]:
# Regla para validar que el campo "NOMCLI" no tenga sílabas compuestas por 2 letras repetidas
REP_LEGAL_Regla_NoSilabasDosLetras = col('REP_LEGAL').rlike(r'\b([a-zA-Z])\1\b')

## Campo FECHA_DE_CONSTITUCION

Regla 9: Campo no debe ser vacío o nulo

In [ ]:
# Regla para validar que el campo "FEC_NACMTO" no sea vacío o nulo
FECHA_DE_CONSTITUCION_Regla_NoVacio = (
    col('FECHA_DE_CONSTITUCION').isNull() | (col('FECHA_DE_CONSTITUCION') == '')
)

Regla 10: No debe ser mayor a la fecha actual (Que no se una fecha en el futuro)

In [ ]:
# Regla para validar que la fecha de nacimiento no es una fecha futura
from pyspark.sql.functions import current_date, datediff, lit, to_date
FECHA_DE_CONSTITUCION_Regla_NoFuturo = (
    to_date(col('FECHA_DE_CONSTITUCION'), 'd/M/yyyy').cast('date') > current_date()
)

Regla 11: Debe ser mayor a 1900/01/01

In [ ]:
# Regla para validar que la fecha de nacimiento sea mayor a 1900/01/01
# Filtrar los registros donde la fecha NO es mayor que '1900-01-01'
from pyspark.sql.functions import lit
FECHA_DE_CONSTITUCION_Regla_MenorIgualQue1900 = (
    to_date(col('FECHA_DE_CONSTITUCION'), 'd/M/yyyy').cast('date') < lit('1900-01-01').cast('date')
)

Regla 12: (FECHA NACIMIENTO -FECHA ACTUAL)/365 <=120 años)

In [ ]:
from pyspark.sql.functions import col, datediff, current_date

# Regla para validar que la fecha de nacimiento exceda los 120 años
FECHA_DE_CONSTITUCION_Regla_Mayor120Anios = (
    (datediff(current_date(),  to_date(col('FECHA_DE_CONSTITUCION'), 'd/M/yyyy').cast('date')) / 365) >= 120
)

Regla 13: No debe ser mayor a la fecha de ingreso

In [ ]:
from pyspark.sql.functions import col, to_date
FECHA_DE_CONSTITUCION_Regla_NoMayorFechaIngreso=  (
    to_date(col('FECHA_DE_CONSTITUCION'), 'd/M/yyyy').cast('date') > to_date(col('FEC_INGBCO'),'d/M/yyyy').cast('date')
)

Regla 14: Estándar ISO8601Format YYYY-MM-DD (ejemplo: 2019-12-11)

In [ ]:
from pyspark.sql.functions import col, datediff, to_date, regexp_extract
# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FECHA_DE_CONSTITUCION_Regla_FormatoDiaMesAnio= (
    col('FECHA_DE_CONSTITUCION').isNotNull() &  # Asegura que el valor no sea nulo
    (regexp_extract(col('FECHA_DE_CONSTITUCION'), pattern, 0) == '')  # Valida el formato usando una expresión regular
)

## Campo CODIGO_LUGAR_DE_CONSTITUCION

Regla 15: Campo no debe ser vacío o nulo

In [ ]:
# Regla para verificar que el campo "LUGNAC" no sea vacío o nulo
CODIGO_LUGAR_DE_CONSTITUCION_Regla_NoVacioONulo = col('CODIGO_LUGAR_DE_CONSTITUCION').isNull() | (col('CODIGO_LUGAR_DE_CONSTITUCION') == '')

Regla 16: "El código de lugar de nacimiento o constitucion debe pertenecer al rango de opciones validas en parámetros // Códigos que esten dentro de la arhivo ""cliciudad"" de la Finclidat. (Códigos DANE)"

In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODCIU'

print(df_csv.columns)

# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]


CODIGO_LUGAR_DE_CONSTITUCION_Regla_ValorPerteneceListaDane = (
    ~col('CODIGO_LUGAR_DE_CONSTITUCION').isin(valores_primera_columna)
)

['CODPAI', 'CODDEP', 'INDICATIVO', 'CODCIU', 'NOMCIU', 'Codigo Centro Poblado Dane', 'DIRECCIONES', 'TELEFONOS', 'NOMBRE_EMPRESA', 'CARGO', 'ORIGEN DE FONDOS', 'CARACTERES ESPECIALES', 'NOMBRE_PROFESION', 'NOMBRE_PROFESION_SIN_NIVEL_ACADEMICO', 'CIU_BANCO1_ME', 'PAIS_BANCO1_ME', 'EMAIL', 'Codigo Pais Dane', 'Nacionalidad', 'CIIU', 'ISO4217', 'Codigo Departamento']


Regla 17: Sólo Numérico

In [ ]:
# Regla para validar que el campo "LUGNAC" sea numérico
CODIGO_LUGAR_DE_CONSTITUCION_Regla_SoloNumerico = ~col('CODIGO_LUGAR_DE_CONSTITUCION').cast('bigint').isNotNull()

Regla 18: Diferente de 0 a 1

In [ ]:
# Regla para validar que el campo "LUGNAC" sea diferente de 0 y 1
CODIGO_LUGAR_DE_CONSTITUCION_Regla_IgualA01 = (col('CODIGO_LUGAR_DE_CONSTITUCION') == 0) | (col('CODIGO_LUGAR_DE_CONSTITUCION') == 1)

## Campo TIPDOC

Regla 19: Campo no debe ser vacío o nulo

In [ ]:
# Regla para validar que el campo "TIPDOC" no sea vacío ni nulo
TIPDOC_Regla_NuloOVacio_j = col('TIPDOC').isNull() | (col('TIPDOC') == '')

## Campo NIT

Regla 21: Campo no debe ser vacío o nulo

In [ ]:
# Regla para validar que el campo NIT no sea vacío ni nulo
NIT_Regla_NuloOVacio_j = col('NIT').isNull() | (col('NIT') == '')

Regla 22: Valida que el número de documento y tipo de documento estén completos en los dos campos, si uno de los dos campos hace falta se etiqueta como NO VALIDO.

In [ ]:
# Regla para validar que ambos campos estén completos (no sea nulo o vacío)
NIT_Regla_NoCompletosTIPDOC_j= (
    (col('NIT').isNull() | (col('NIT') == '')) |  # Verifica si NIT es nulo o vacío
    (col('TIPDOC').isNull() | (col('TIPDOC') == ''))  # Verifica si TIPDOC es nulo o vacío
)

Regla 23: Si el tipo de documento es Cedula de ciudadanía se valida que el número de documento sea de 6, 7 u 8 dígitos y puede comenzar con cualquier valor numérico


In [ ]:
# Regla para validar el campo NIT si TIPDOC es igual a "CC" pero el número de documento no tiene 6, 7 u 8 dígitos
NIT_Regla_Tipdoc_CC_NoTieneDigitosAdecuados_j = (
     # Verifica si TIPDOC es igual a "CC"
    (~col('NIT').rlike('^\d{6}$') &  # Verifica que el NIT no tenga 6 dígitos
     ~col('NIT').rlike('^\d{7}$') &  # Verifica que el NIT no tenga 7 dígitos
     ~col('NIT').rlike('^\d{8}$') &  # Verifica que el NIT no tenga 8 dígitos
     ~col('NIT').rlike('^\d{9}$') &
     ~col('NIT').rlike('^\d{10}$')
     )  
)

Regla 24: Si el tipo de documento es Cedula de ciudadanía se valida que el número de documento sea de 10 dígitos siempre y cuando éste comience por el digito 1.


In [ ]:
# Filtro para seleccionar los registros con TIPDOC "CC", el primer valor del NIT es '1', pero no tiene 10 dígitos
NIT_Regla_Tipdoc_CC_PrimerDigito1_NoTiene10Digitos_j = (
    (col('TIPDOC') == '1') &  # Verifica si TIPDOC es igual a "CC"
    col('NIT').rlike('^1') &  # Verifica si el primer valor del NIT es '1'
    ~col('NIT').rlike('^1\d{9}$')  # Verifica que el NIT no tenga 10 dígitos si comienza con "1"
)

Regla 26: Si Tipo de Identificacion es 'TI', el dato debe contener (11) caracteres númericos ó (10) caracteres alfanúmericos


In [ ]:

NIT_Regla_Tipdoc_TI_PatronEspecifico_siInicia1_j = (
    (col('TIPDOC') == '4') &  # Verifica si TIPDOC es igual a "TI" (Tarjeta de Identidad)
    (col('NIT').rlike('^1')) &  # Verifica si el NIT inicia con '1'
    (~col('NIT').rlike('^1\d{9,10}$'))  # Verifica la longitud total del NIT
)

Regla 28:  Si el tipo de documento es Registro Civil puede contener 3 letras de la A-Z al inicio del número, seguido de un numero entre 7,8,9 o 10 dígitos.


In [ ]:
# Regla para validar el campo NIT si TIPDOC es igual a "Registro Civil"
NIT_Regla_Tipdoc_RegistroCivil_NoCumple_j =  (
    (col('TIPDOC') == '6') &  # Verifica si TIPDOC es igual a "Registro Civil"
    ~(col('NIT').rlike('^[A-Za-z0-9]{1,3}[0-9]{5,10}$'))  # Verifica el patrón del NIT modificado
)

Regla 29: Si Tipo de Identificacion es 'CE', el dato debe contener (6) caracteres alfanúmericos.


In [ ]:
NIT_Regla_Longitud_CE_NoCumpleCondicion_j = (
    (col('TIPDOC') == '2') &  # Verifica si TIPDOC es igual a "CE"
    ~(col('NIT').rlike('^[0-9A-Za-z]{6}$'))  # NIT contiene exactamente 6 caracteres alfanuméricos
)

Regla 30: Si el tipo de documento es cedula de Extranjería, puede tener la letra E al inicio, seguido de un digito de 1 a 9 y continuar con dígitos de 0 a 9 con una longitud total de 5, 6 o 7 dígitos, para un total máximo de 9 caracteres


In [ ]:
NIT_Regla_Tipdoc_CedulaExtranjeria_NoPatron_j = (
    (col('TIPDOC') == '2') &  # Verifica si TIPDOC es igual a "Cédula de Extranjería"
    col('NIT').rlike('^[A-Za-z0-9][1-9][0-9]{4,6}$') &  # Verifica el patrón del NIT modificado
    (~(length(col('NIT')) >= 7) | ~(length(col('NIT')) <= 9))  # Regla para longitud total
)

Regla 32: Si el tipo de documento es Pasaporte el número de documento puede tener o no dos letras al inicio y continuar con dígitos de 0 a 9 con una longitud total de 6, 7, 8, 9 o 10 dígitos


In [ ]:
NIT_Regla_Tipdoc_Pasaporte_NoCumplePatron_j = (
    (col('TIPDOC') == '5') &  # Verifica si TIPDOC es igual a "Pasaporte"
    ~(col('NIT').rlike('^(?:[A-Z]{2})?\d{6,10}$'))  # Verificación del patrón específico del NIT para Pasaporte
)

Regla 33: Si el tipo de documento es NIT el número de documento debe comenzar por 8 o 9, seguido de 8 dígitos que pueden estar entre 0,9 y terminar con el digito de verificación.


In [ ]:
# Filtro para registros donde TIPDOC es igual a "NIT" pero el campo NIT no sigue el patrón específico
NIT_Regla_Tipdoc_NIT_NoCumplePatron_j = (
    (col('TIPDOC') == '3') &  # Verifica si TIPDOC es igual a "NIT"
    ~(col('NIT').rlike('^[89]\\d{8}'))  # Verificación del patrón específico del NIT para NIT
  #  ~(col('NIT').rlike('^[89]\\d{8}'-\\$d')) Verificacion de digito 
)

Regla 34: Permiso de Permanencia (PP), debe tener una longitud de 15 dígitos y todos deberán ser numéricos.


In [ ]:
NIT_Regla_Tipdoc_PP_NoCumplePatron_j = (
    (col('TIPDOC') == '9') &  # Verifica si TIPDOC es igual a "PP"
    ~(col('NIT').rlike('^\\d{15}$'))  # Verificación de la longitud numérica del NIT para Permiso de Permanencia
)

Regla 35: Si Tipdoc = 7 entonces debe ser de longitud 9 y Debe ir 600.0000.000 a 799.999.999


In [ ]:
# Filtro para registros donde TIPDOC es igual a "7" pero no cumple con la condición dada
NIT_Regla_Tipdoc_7_NoCumpleCondicion_j = (
    (col('TIPDOC') == '7') &  # Verifica si TIPDOC es igual a "7"
    ~((col('NIT').rlike('^[0-9]{9}$')) &  # NIT contiene exactamente 9 caracteres numéricos
      (col('NIT').between(600000000, 799999999)))  # NIT está en el rango especificado
)

Regla 37: Si Tipo de Identificacion es 'PA', el dato debe contener Minimo (6) y Maximo(16) caracteres alfanúmericos.


In [ ]:
NIT_Regla_Longitud_PA_NoCumpleCondicion_j = (
    (col('TIPDOC') == '5') &  # Verifica si TIPDOC es igual a "PA"
    ~(col('NIT').rlike('^[0-9A-Za-z]{6,16}$'))  # NIT contiene entre 6 y 16 caracteres alfanuméricos
)

Regla 44: Campo debe ser único en la combinación tipo de identificación + nitcli


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

def filtro_registros_duplicados_nit_tipdoc(df_spark: DataFrame) -> 'Column':
    # Cálculo de registros con duplicados en la combinación de 'NIT' y 'TIPDOC'
    NIT_Regla_unicidadCombinacionNitMasTipo = (
        df_spark
        .groupBy('NIT', 'TIPDOC','FechaCorte')
        .agg(F.count('*').alias('conteo'))
        .filter("conteo > 1")
    )

    # Obtener una lista de 'NIT' que tienen duplicados en la combinación de 'NIT' y 'TIPDOC'
    nits_con_duplicados = NIT_Regla_unicidadCombinacionNitMasTipo.select('NIT').collect()

    # Convertir la lista de 'NIT' en una lista de valores para filtrar
    lista_nits_con_duplicados = [row['NIT'] for row in nits_con_duplicados]

    # Aplicar el filtro para seleccionar aquellos registros cuyo 'NIT' está en la lista de duplicados
    filtro_unicidad = col('NIT').isin(lista_nits_con_duplicados)

    return filtro_unicidad


# Llamar a la función para obtener el filtro
NIT_Regla_unicidadCombinacionNitMasTipo_j = filtro_registros_duplicados_nit_tipdoc(df_spark)

Regla 45: Para el tipo CC la longitud no puede ser superior a 10 caracteres


In [ ]:
NIT_Regla_Tipdoc_CC_LongitudExcedida_j =  (
    (col('TIPDOC') == '1') &  # Verifica si TIPDOC es igual a "CC"
    (col('NIT').rlike('^\d{11,}$'))  # NIT contiene más de 10 caracteres numéricos
)

## Campo FECEXPREP

Regla 45: Campo no debe estar vacío

In [ ]:
# Regla para validar que el campo FECEXP no esté vacío
FECEXPREP_Regla_NoVacio = (col('FECEXPREP').isNull() | (col('FECEXPREP') == ''))

Regla 46: Campo no texto

In [ ]:
FECEXPREP_Regla_NoEsTexto = (
    col('FECEXPREP').rlike(r'[^\d/]')
)

Regla 47: (FECHA expedición -FECHA ACTUAL)/365 <=120 años)

In [ ]:
from pyspark.sql.functions import col, datediff, current_date

# Regla para validar que la fecha de nacimiento exceda los 120 años
FECEXPREP_Regla_DiferenciaMayorIgual120 = ((datediff(current_date(), to_date(col('FECEXPREP'), 'd/M/yyyy').cast('date') ) / 365) >= 120)

Regla 48: Debe ser mayor a 19000101

In [ ]:
from pyspark.sql.functions import to_date, col

FECEXPREP_Regla_FechaAnterior1900  = ~(
    to_date(col('FECEXPREP'), 'd/M/yyyy').cast('date') > lit('1900-01-01').cast('date')
)

## Campo CIUDAD_EXPEDICION

Regla 49: Campo no debe estar vacío o nulo


In [ ]:
# Regla para verificar que el campo "CIUDAD_EXPEDICION" no esté vacío o nulo
CIUDAD_EXPEDICION_Regla_NoVacioONulo = col('CIUDAD_EXPEDICION').isNull() | (col('CIUDAD_EXPEDICION') == '')

Regla 50: Diferente de 0 a 1

In [ ]:
# Regla para verificar que el campo "CIUDAD_EXPEDICION" sea diferente de 0 y 1
CIUDAD_EXPEDICION_Regla_No0o1 = (col('CIUDAD_EXPEDICION') == 0) | (col('CIUDAD_EXPEDICION') == 1)

Regla 51: Solo Numerico

In [ ]:
# Regla para verificar que el campo "CIUDAD_EXPEDICION" sea solo numérico
CIUDAD_EXPEDICION_Regla_SoloNumerico = ~col('CIUDAD_EXPEDICION').cast('string').rlike('^[0-9]+$')

Regla 52: El código de lugar de nacimiento o constitucion debe pertenecer al rango de opciones validas en parámetros

In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODCIU'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

CIUDAD_EXPEDICION_Regla_ValorPerteneceListaDane = (
    ~col('CIUDAD_EXPEDICION').isin(valores_primera_columna)
)

## Campo DIRECCION_COMERCIO

Regla 53 Campo no debe estar vacío o nulo


In [ ]:
# Regla para verificar que el campo "DIRECCION_COMERCIO" no esté vacío o nulo
DIRECCION_COMERCIO_Regla_NoVacioONulo = col('DIRECCION_COMERCIO').isNull() | (col('DIRECCION_COMERCIO') == '')

Regla 54 No se aceptan caracteres especiales a excepción de ( # - .) (Numeral, punto y guion medio) // No debe contener solo caracteres especiales ej:  guiones, “#”, “/”, entre  otros.


In [ ]:
# Regla para verificar que el campo "DIRECCION_COMERCIO" no contiene caracteres especiales no permitidos
DIRECCION_COMERCIO_Regla_NoCaracteresEspeciales = col('DIRECCION_COMERCIO').rlike(r'^[^#\.\-\w]+$')

Regla 55 La DIRECCION RESIDENCIA no debe estar en la Lista de Datos errados


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'DIRECCIONES'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_segunda_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

DIRECCION_COMERCIO_Regla_ValorListaErrados = (
    ~col('DIRECCION_COMERCIO').isin(valores_segunda_columna)
)

Regla 56 No debe ser solo numérico


In [ ]:
# Regla para verificar que el campo "DIRECCION_COMERCIO" no sea solo numérico
DIRECCION_COMERCIO_Regla_NoSoloNumeros = col('DIRECCION_COMERCIO').rlike('^[0-9]+$')

Regla 57  No debe contener solo una palabra ej “avenida”; “calle”


In [ ]:
from pyspark.sql.functions import col, size, split

# Regla para verificar que el campo "DIRECCION_COMERCIO" no contenga solo una palabra
DIRECCION_COMERCIO_Regla_NoSoloUnaPalabra = (
    size(split(col('DIRECCION_COMERCIO'), ' ')) < 2  # Verifica si hay al menos dos palabras en el campo
)

Regla 58 No debe contener palabras como “no aplica”; “NO APLICA”


In [ ]:
from pyspark.sql.functions import lower
# Regla para verificar si el campo "DIRECCION_COMERCIO" contiene la palabra "no aplica"
DIRECCION_COMERCIO_Regla_ContieneNoAplica = lower(col('DIRECCION_COMERCIO')).contains('no aplica')

Regla 59 La DIRECCION RESIDENCIA no debe ser <= a 7 caracteres


In [ ]:
# Regla para verificar que la longitud del campo "DIRECCION_COMERCIO" sea mayor que 7 caracteres
DIRECCION_COMERCIO_Regla_LongitudMinima = length(col('DIRECCION_COMERCIO')) <= 7

## Campo CODIGO_CIUDAD_DE_SEDE_PRINCIPAL

Regla 60 Campo no debe ser vacío o nulo


In [ ]:
# Regla para verificar que el campo "CODIGO_CIUDAD_DE_SEDE_PRINCIPAL" no sea vacío ni nulo
CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_NoVacioNiNulo =(col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').isNull() | (col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL') == '')) 

Regla 61 Campo puede estar vacío, si no tiene diligenciado el campo Dirección de Residencia y Barrio de Residencia


In [ ]:
CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_NoVacioNiNulo_SinDireccionBarrioDiligenciado =(col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').isNull() | (col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL') == '')) & ((col('DIRECCION_COMERCIO').isNotNull() | (col('DIRECCION_COMERCIO') != '')))

Regla 62 Debe ser numérico


In [ ]:
# Regla para verificar si el campo "CODIGO_CIUDAD_DE_SEDE_PRINCIPAL" es numérico
CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_EsNumerico = col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').cast('bigint').isNull()

Regla 63 Códigos de ciudad parametrizados DANE


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODCIU'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_ValorListaDane = (
    ~col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').isin(valores_primera_columna)
)

Regla 64 No debe estar definida con el codigo 0


In [ ]:
CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_ValorCodigoCero = (
    col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').isin("0")
)

## Campo TEL_REP

Regla 65: Campo no debe ser vacío o nulo


In [ ]:
# Regla para verificar que el campo "TEL_REP" no es nulo ni vacío
TEL_REP_Regla_NoNuloOVacio = col('TEL_REP').isNull() | (col('TEL_REP') == '')

Regla 66 El campo debe ser numérico


In [ ]:
# Regla para verificar que el campo "TEL_REP" es numérico
TEL_REP_Regla_Numerico = ~col('TEL_REP').cast('bigint').rlike('^[0-9]+$')

Regla 67 Debe tener 10 Caracteres


In [ ]:
# Regla para validar que el campo "TEL_REP" tiene exactamente 10 caracteres
from pyspark.sql.functions import length, col

TEL_REP_Regla_Longitud10 = (length(col('TEL_REP')) != 10)


Regla 68 El registro no debe ser menor al numero 999999999


In [ ]:
# Regla para validar que el valor en "TEL_REP" no sea menor a "999999999"
#TEL_REP_Regla_NoMenorA999M = col('TEL_REP').cast('bigint') <= 999999999

Regla 68 "El valor registrado en el campo debe ser mayor a ""1000000000"""


In [ ]:
# Regla para validar que el valor en "TEL_REP" es mayor a "1000000000"
#TEL_REP_Regla_MayorA1Billion = col('TEL_REP').cast('bigint') < 1000000000

Regla 70 Debe empezar por 30 -  31 - 32 - 35 (parametrizable) si el país es Colombia


In [ ]:
#TEL_REP_Regla_IniciaConParametroColombia = (col('NOM_PAIS_CONSTITUCIÓN') == '169') & ~(
#    (col('TEL_REP').startswith('30')) |
#    (col('TEL_REP').startswith('31')) |
 #   (col('TEL_REP').startswith('32')) |
  #  (col('TEL_REP').startswith('35'))
#)

Regla 71  El campo no debe contener  una sola letra minúscula o mayúscula


In [ ]:
# Regla para validar que "TEL_REP" no contenga una sola letra
TEL_REP_Regla_NoUnaSolaLetra = col('TEL_REP').rlike(r'^[a-zA-Z]$')

Regla 72 No debe contener oraciones como “no aplica”; “NO APLICA”


In [ ]:
# Regla para validar que "TEL_REP" no contenga las oraciones "no aplica" o "NO APLICA"
TEL_REP_Regla_NoNoAplica = col('TEL_REP').rlike(r'\b(?:no\s*aplica|NO\s*APLICA)\b')

Regla 73 No texto


In [ ]:
# Regla para validar que "TEL_REP" no contiene texto
TEL_REP_Regla_NoTexto = col('TEL_REP').rlike(r'^[a-zA-Z]$')

Regla 74 No estar en la lista de Datos errados


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_tercera_columna = 'TELEFONOS'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_tercera_columna = [row[0] for row in df_csv.select(nombre_tercera_columna).collect()]

TEL_REP_Regla_ValorListaErrados = (
    ~col('TEL_REP').isin(valores_tercera_columna)
)

## Campo TEL_COMERC

Regla 75 Es obligatorio sólo si no tiene TEL_REP


In [ ]:
TEL_COMERC_Regla_VacioNuloSinTEL_REP = (col('TEL_REP').isNull() | (col('TEL_REP') == '')) & (col('TEL_COMERC').isNull() | (col('TEL_COMERC') == ''))

Regla 76 Campo puede estar vacío, si no tiene diligenciado el campo Dirección Comercial y CODIGO_CIUDAD_DE_SEDE_PRINCIPAL


In [ ]:
TEL_COMERC_Regla_vaciosConDirecciones = (
    (col('TEL_COMERC').isNull() | (col('TEL_COMERC') == '')) &
    ~(col('DIRECCION_COMERCIO').isNull() | (col('DIRECCION_COMERCIO') == '')) &
    ~(col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL').isNull() | (col('CODIGO_CIUDAD_DE_SEDE_PRINCIPAL') == ''))
)

Regla 77 No se permiten en el registro ningún carácter diferente a un numero


In [ ]:
# Regla para validar que "TEL_COMERC" contenga solo números
TEL_COMERC_Regla_SoloNumeros = col('TEL_COMERC').rlike('[^0-9]')

Regla 78 Debe ser mayor 7 digitos. 


In [ ]:
# Regla para validar la longitud del campo "TEL_COMERC" entre 7 y 10 caracteres
TEL_COMERC_Regla_Longitud = (
    (length(col('TEL_COMERC')) < 7) | (length(col('TEL_COMERC')) > 10)
)

Regla 79 El registro no debe ser menor al numero 999999


In [ ]:
# Regla para validar si el campo "TEL_COMERC" no es menor a 999999
TEL_COMERC_Regla_NoMenorQue999999 = (
    col('TEL_COMERC') < 999999
)

Regla 80 "el valor registrado en el campo debe ser mayor a ""1000000"""


In [ ]:
# Regla para validar si el campo "TEL_COMERC" es mayor a 1000000
TEL_COMERC_Regla_MayorQue1000000 = (
    col('TEL_COMERC') < 1000000
)

Regla 81 Los teléfonos locales suministrado por los titulares deben contener el siguiente patrón: NXX-XXXX (siete caracteres numéricos), donde N son números del 2 al 8. Los números donde N es 9 están reservados para teléfonos públicos y X es cualquier número del 0 al 9.


In [ ]:
# Regla para validar el patrón de números telefónicos
TEL_COMERC_Regla_Patron = (
    ~(col('TEL_COMERC').rlike(r'^[2-8]\d{6}$|^[2-8]\d{9}$'))
)

#Esta regla utiliza la expresión regular ^[2-8][0-9]{2}-[0-9]{4}$ para verificar si el número de teléfono tiene el patrón NXX-XXXX, 
# donde N es un número del 2 al 8, y X es cualquier dígito del 0 al 9.

Regla 82  El campo no debe contener  una sola letra minúscula o mayúscula


In [ ]:
# Regla para verificar que el campo 'TEL_COMERC' no contiene una sola letra minúscula o mayúscula
TEL_COMERC_Regla_NoLetraSola = col('TEL_COMERC').rlike(r'\b[A-Za-z]\b')

Regla 83 No debe contener silabas con letras combinadas o repetidas de cualquier forma en minúscula o mayúscula


In [ ]:
# Regla para verificar que el campo 'TEL_COMERC' no contiene sílabas con letras combinadas o repetidas
TEL_COMERC_Regla_NoSilabasRepetidas = (
    col('TEL_COMERC').rlike(r'([a-zA-ZáéíóúÁÉÍÓÚ]{2,})\1') &
    # Regla para verificar que el campo 'TEL_COMERC' no contiene letras
    col('TEL_COMERC').rlike(r'[a-zA-ZáéíóúÁÉÍÓÚ]')
)

Regla 84 No debe contener oraciones como “no aplica”; “NO APLICA”


In [ ]:
# Regla para verificar que el campo 'TEL_COMERC' no contiene las frases "no aplica" o "NO APLICA"
TEL_COMERC_Regla_NoContieneNoAplica = (
    col('TEL_COMERC').rlike(r'\b(?:no aplica|NO APLICA)\b')
)

## Campo NOM_PAIS_CONSTITUCIÓN

Regla 85 Campo no debe estar vacío o nulo


In [ ]:
NOM_PAIS_CONSTITUCIÓN_Regla_NoVacioNulo = col('NOM_PAIS_CONSTITUCIÓN').isNull() | (col('NOM_PAIS_CONSTITUCIÓN') == '')

Regla 86 validar estándar ISO3166


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = "Codigo Pais Dane"


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

NOM_PAIS_CONSTITUCIÓN_Regla_ValorListaDane = (
    ~col('NOM_PAIS_CONSTITUCIÓN').isin(valores_primera_columna)
)

## Campo NOMBRE

Regla 87 Campo no debe contener números


In [ ]:
NOMBRE_Regla_NoNumeros = (
    col('NOMBRE').rlike(r'\d')  # Verifica que no haya ningún dígito numérico
)

Regla 88 Campo no debe contener caracteres especiales (a excepción de espacios)


In [ ]:
# Regla para validar que "NOMBRE" no contenga caracteres especiales, excepto espacios
NOMBRE_Regla_NoCaracteresEspeciales = (
    col('NOMBRE').rlike(r'[^\w\sáéíóúüÁÉÍÓÚÜ]') &
    ~col('NOMBRE').rlike(r'[ .ñÑ]')
)

Regla 89 Que no exista solo un punto (.), coma (,), asteriscos (*)


In [ ]:
# Regla para validar que "NOMBRE" no contenga solo un punto, coma o asterisco
NOMBRE_Regla_NoSoloPuntoComaAsterisco = (
    col('NOMBRE').rlike(r'^[,*]$')  # No contiene solo un punto, coma o asterisco
)

Regla 90 No debe contener puntos, comas


In [ ]:
NOMBRE_Regla_SinPuntosComas = (
    col('NOMBRE').rlike(r',')  # Valida solo las comas
)

Regla 91 No debe contener sucesión del mismo carácter más de 3 veces (XXXX, AAAAAA,etc.)


In [ ]:
# Regla para validar que "NOMBRE" no contenga una secuencia repetida de un mismo carácter más de tres veces
NOMBRE_Regla_NoSecuenciaRepetida = (
    col('NOMBRE').rlike(r'[^ ](.)\1{3,}')  # Excluye secuencias repetidas de un mismo carácter más de tres veces, excepto espacios
)

Regla 92 Si contiene solo números, si contiene solo una sola letra minúscula o mayúscula


In [ ]:
# Regla para validar que "NOMBRE" no contenga solo números ni una sola letra minúscula o mayúscula
NOMBRE_Regla_NoSoloNumerosLetras = (
    col('NOMBRE').rlike(r'^\d+$') |  # No contiene solo números
    col('NOMBRE').rlike(r'^[a-zA-Z]$')  # No contiene solo una letra minúscula o mayúscula
)

Regla 93 No debe estar en la lista de errados


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'NOMBRE_EMPRESA'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

NOMBRE_Regla_NoListaErrados = (
    col('NOMBRE').isin(valores_primera_columna)
)

## Campo Dir_Representante

Regla 94 No se aceptan caracteres especiales a excepción de ( # - .) (Numeral, punto y guion medio) // No debe contener solo caracteres especiales ej:  guiones, “#”, “/”, entre  otros.


In [ ]:
# Regla para validar que "Dir_Representante" no tenga caracteres especiales no permitidos y no sea solo caracteres especiales
Dir_Representante_Regla_CaracteresEspeciales = (
    ~((col('NOMBRE').isNull()) | (col('NOMBRE') == '')) &
    (
        col('Dir_Representante').rlike(r'^[^a-zA-Z0-9#\-\.]+$') |  # Verifica que no contenga solo caracteres especiales
        col('Dir_Representante').rlike(r'^[^\w\s#\-\.]+$')  # Verifica que no contenga caracteres especiales excepto (#, -, .)
    )
)


Regla 95 No debe ser solo numérico


In [ ]:
# Regla para verificar que "Dir_Representante" no sea solo numérico
Dir_Representante_Regla_NoSoloNumerico = (
    col('Dir_Representante').rlike('^[0-9]+$')
)

Regla 96 El campo no este diligenciado solo con silabas en minúscula o mayúscula de una a cuatro letras combinadas en cualquier orden


In [ ]:
# Regla para verificar que "Dir_Representante" no esté diligenciado solo con silabas en minúscula o mayúscula de una a cuatro letras combinadas en cualquier orden
Dir_Representante_Regla_NoSilabasEspecíficas = (
    col('Dir_Representante').rlike(r'^(?=.*[a-zA-Z])(?!.*\d)(?!.*[-!$%^&*()_+|~=`{}\[\]:";<>?,.\/])(?=.*[aeiou]{1,4})[a-zA-Z]{1,4}$')
)
#Reorna los registros que cumplan lo siguiente:
#Contiene al menos una letra.
#No contiene dígitos.
#No contiene caracteres especiales, excepto letras y números.
#Contiene entre 1 y 4 vocales.
#Tiene entre 1 y 4 letras en total.

Regla 97  No debe contener solo una palabra ej “avenida”; “calle”


In [ ]:
from pyspark.sql.functions import col, split, size
# Regla para verificar que "Dir_Representante" no contenga solo una palabra
Dir_Representante_Regla_NoSoloUnaPalabra = (
    (~col('Dir_Representante').isNull()) &  # Verifica que no sea nulo
    ~(col('Dir_Representante') == '') &    # Verifica que no esté vacío
    (size(split(col('Dir_Representante'), ' ')) < 2)  # Verifica si hay al menos dos palabras en el campo
)

Regla 98 No debe contener palabras como “no aplica”; “NO APLICA”


In [ ]:
from pyspark.sql.functions import lower
# Regla para verificar que "Dir_Representante" no contenga las palabras "no aplica" o "NO APLICA"
Dir_Representante_Regla_ContieneNoAplica = (
    lower(col('Dir_Representante')).rlike(r'\bno\s*aplica\b')  # Verifica la presencia de "no aplica" en cualquier caso
)

Regla 99 Si contiene solo números, si contiene solo una sola letra minúscula o mayúscula


In [ ]:
# Regla para verificar que "TOT_ACTIVOB" no contenga solo números o una sola letra
Dir_Representante_Regla_NoSoloNumerosNiUnaLetra = (
    col('Dir_Representante').rlike(r'^\d+$|^[a-zA-Z]$')  # Verifica que no contenga solo números o una sola letra
)

Regla 100 La DIRECCION RESIDENCIA no debe ser <= a 7 caracteres


In [ ]:
# Regla para verificar que "Dir_Representante" tenga una longitud mayor a 7 caracteres
Dir_Representante_Regla_LongitudMayorA7 = (
    length(col('Dir_Representante')) <= 7  # Verifica si la longitud es mayor a 7 caracteres
)

## Campo CIU_REP

Regla 101 Códigos de ciudad parametrizados DANE


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODCIU'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

CIU_REP_Regla_ValorListaDane = (
    ~col('CIU_REP').isin(valores_primera_columna)
)

## Campo CIIU

Regla 102 Debe estar diligenciado (no presenten valor en cero  0  o NULL).


In [ ]:
CIIU_Regla_NoCeroONuloOVacio = (
    col('CIIU').isNull() |  # Verifica que "CIIU" no sea nulo
    (col('CIIU') == '') |     # Verifica que "CIIU" no esté vacío
    (col('CIIU') == 0)        # Verifica que "CIIU" no sea cero
)

Regla 103 Se debería validar el código enviado con el que se encuentre parametrizado en Golden Record. (Debe ser la versión actualizada DANE, v4)


In [ ]:
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CIIU'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]


CIIU_Regla_ValorListaCIIU = (
    ~col('CIIU').isin(valores_primera_columna)
)

## Campo TOT_VENTASB

Regla No debe contener ceros

In [ ]:
TOT_VENTASB_Regla_Cero = (
    col('TOT_VENTASB').isin(0)  # Verifica que "TOT_VENTASB" sea 0
)

Regla 104 No deben estar en 0 (cero), nulo, 1 (uno)

In [ ]:
# Regla para verificar que "TOT_VENTASB" no sea 0, 1, nulo o vacío
TOT_VENTASB_Regla_NoUnoNuloOVacio = (
    col('TOT_VENTASB').isin(1) |  # Verifica que "TOT_VENTASB" no sea 0 ni 1
    col('TOT_VENTASB').isNull() |  # Verifica que "TOT_VENTASB" no sea nulo
    (col('TOT_VENTASB') == '')       # Verifica que "TOT_VENTASB" no esté vacío
)

Regla 105 "Si el campo "" Descrip_Actividad_Economica"" por “Asalariado”, ""Pensionado"", y ""Empleado Público"", se debe validar que el campo ""Ingreso"" esté diligenciado (no vacio)"


In [ ]:
# Regla para verificar que "TOT_VENTASB" no es vacío o nulo si "ACT_ECNCA" tiene ciertos valores
TOT_VENTASB_Regla_NoVacioConActividadesEspecificas = (
    (
        (col('CIIU').isin(10, 20, 8423, 7511, 7512, 7515, 8423)) &
        (col('TOT_VENTASB').isNull())  # Verifica que "TOT_VENTASB" no sea vacío o nulo
    ) | (
        (col('CIIU').isin(10, 20, 8423, 7511, 7512, 7515, 8423)) &
        (col('TOT_VENTASB') == '')  
    )
)

Regla 106 "Si el campo "" Descrip_Actividad_Economica"" por “Asalariado”, ""Pensionado"", y ""Empleado Público"", se debe validar que el campo ""Ingreso"" esté diligenciado con números (no texto- ni caracteres especiales)"


In [ ]:
# Regla para verificar que "TOT_VENTASB" contiene solo números si "ACT_ECNCA" tiene ciertos valores
TOT_VENTASB_Regla_NumeroConActividadesEspecificas = (
    col('CIIU').isin(10, 20, 8423, 7511, 7512, 7515, 8423) &
    col('TOT_VENTASB').cast('float').isNull()  # Verifica que 'TOT_VENTASBB' no consista solo en números
)
  

Regla 107 Campo numérico


In [ ]:
TOT_VENTASB_Regla_EsNumerico = (
    ~col('TOT_VENTASB').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$')
)

Regla 108 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún ingreso


In [ ]:
# Valor del SMLV
#smlv = 1160000

# Regla para verificar que "TOT_VENTASB" no sea menor al 10% del SMLV
TOT_VENTASB_Regla_NoMenorQueCienMil = (
    col('TOT_VENTASB') <= 0.1  # Verifica que "TOT_VENTASB" sea al menos el 10% del SMLV
)

Regla 109 "Si el campo "" Descrip_Actividad_Economica"" por “Asalariado”, ""Pensionado"", y ""Empleado Público"", el valor registrado en el campo debe ser mayor al SMLV (salario mínimo legal vigente- campo parametrizable) y menos a 100000002"


In [ ]:
# Valor del SMLV
#smlv = 1160000  # Puedes cambiar este valor según el SMLV actual

# Regla para validar el campo "TOT_VENTASB" con condiciones sobre "ACT_ECNCA", es decir, sobre el codigo CIIU
#TOT_VENTASB_Regla_mayorSMLV_menor100M = (
   # (
    #    (col('CIIU').isin(10, 20, 8423, 7511, 7512, 7515, 8423)) &
    #    (col('TOT_VENTASB') < smlv)  # Si 'TOT_VENTASB' es menor que el valor de SMLV
    #) | (
    #    (col('CIIU').isin(10, 20, 8423, 7511, 7512, 7515, 8423)) &
    #    (col('TOT_VENTASB') > 100000000)  # Si 'TOT_VENTASB' es mayor que 100 millones
    #)
#)

## Campo OTR_INGRSOB

Regla 110 Campo numérico


In [ ]:
# Regla para verificar si el campo "OTR_INGRSOB" es numérico
OTR_INGRSOB_Regla_EsNumerico = (
    col('OTR_INGRSOB').isNotNull() &  # Verifica que el campo no sea nulo
    (col('OTR_INGRSOB') != '') &  # Verifica que el campo no esté vacío
    ~(col('OTR_INGRSOB').cast('string').\
        rlike('^\\d+(,\\d+)?$'))  # Verifica que el campo no sea numérico con coma como separador decimal
)

Regla 111 "El campo otro ingreso se calificará como “cumple” si está vacio  o contener el caracter especial guion (-), una vez o varias veces (------), de igual forma puede contener silabas como ""na"", ""NA"", o ""no aplica"" , o ""NO APLICA"""


In [ ]:
# Regla para validar condiciones específicas en un campo, permitiendo que esté vacío
from pyspark.sql.functions import col

OTR_INGRSOB_Regla_CumpleCondiciones = (
    (col('OTR_INGRSOB') != '0') &  # No es cero
    (col('OTR_INGRSOB') != '') &  # No es vacío
    (~col('OTR_INGRSOB').rlike(r'-+')) &  # No contiene el carácter guion (-) una o varias veces
    (~col('OTR_INGRSOB').rlike(r'^-?\d+(\.\d+)?$')) &  # No es un valor numérico
    (~col('OTR_INGRSOB').rlike(r'\b(?:na|NA|no aplica|NO APLICA)\b'))  # No contiene palabras específicas
)

Regla 112 No debe contener silabas o letras en minúscula o mayúscula combinadas en cualquier orden


In [ ]:
# Regla para validar que "OTR_INGRSOB" no contenga texto
OTR_INGRSOB_Regla_NoTexto = col('OTR_INGRSOB').isNotNull() & col('OTR_INGRSOB').cast("float").isNull()

## Campo DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES

Regla 113 Si el campo “Otro_Ingreso” para valores mayores a cero, en estos casos el campo “descripcion_otros_ingresos” debe estar diligenciado solo con texto


In [ ]:
# Definir una expresión regular para verificar que DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES no contenga dígitos
pattern = r'\D+'  # Esta expresión regular seleccionará cualquier secuencia que no sea un dígito

DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES_Regla_TextoSinDigitos = (
    col('DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES').isNotNull() &  # Asegura que DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES no sea nulo
    (col('OTR_INGRSOB') > 0) &  # Verifica que OTR_INGRSO sea mayor que cero
    ~(regexp_extract(col('DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES').cast("string"), pattern, 0) != '')  # Verifica que DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES no contenga dígitos
)

## Campo TOT_GASTOSB

Regla no Cero

In [ ]:
TOT_GASTOSB_Regla_Cero = (
    col('TOT_GASTOSB').isin(0)  # Verifica que "TOT_GASTOSB" sea 0
)

Regla 114 El campo debe ser numérico


In [ ]:
# Regla para validar que "TOT_GASTOSB" sea numérico
TOT_GASTOSB_Regla_SerNumerico =  (
   ~col('TOT_GASTOSB').rlike('^-?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?$')
)

Regla 115 No deben estar en 0 (cero), nulo, 1 (uno)


In [ ]:
# Regla para validar que "TOT_GASTOSB" no sea 0, 1, nulo o vacío
TOT_GASTOSB_Regla_NoUnoNuloVacio = (
    col('TOT_GASTOSB').isNull() |               # Verifica que "TOT_GASTOSB" no sea nulo
    (col('TOT_GASTOSB') == '') |                  # Verifica que "TOT_GASTOSBSBSB" no esté vacío                  # Verifica que "TOT_GASTOSB" no sea igual a 0
    (col('TOT_GASTOSB') == 1)                      # Verifica que "TOT_GASTOSB" no sea igual a 1
)

Regla 116 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún egreso


In [ ]:
# Valor del Salario Mínimo Mensual Legal Vigente (SMMLV)
#SMMLV = 1160000  # Por ejemplo, el SMMLV en Colombia es 1.160.000 pesos

# Calcular el 10% del SMMLV
limite_egreso = 0.1  # 10% del SMMLV

# Regla para validar que "TOT_GASTOSB" no esté por debajo del 10% del SMMLV
TOT_GASTOSB_Regla_NoMenorQueCienMil = (
    col('TOT_GASTOSB') <= limite_egreso  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

## Campo TOT_PASIVOB


Regla 117 Debe ser mayor a 0


In [ ]:
# Regla para validar que "VALOR_PASIVOS" sea mayor que cero
TOT_PASIVOB_Regla_MayorQueCero = (
    col('TOT_PASIVOB') < 0  # Verifica que el valor sea mayor que cero
)

Regla 118 No deben estar en nulo o 1 (uno)


In [ ]:
# Regla para validar que "VALOR_PASIVOS" no sea 0, 1, nulo o vacío
TOT_PASIVOB_Regla_NoUnoNuloVacio = (
    col('TOT_PASIVOB').isNull() |               # Verifica que "VALOR_PASIVOS" no sea nulo
    (col('TOT_PASIVOB') == '') |                  # Verifica que "VALOR_PASIVOS" no esté vacío
    (col('TOT_PASIVOB') == 1)                      # Verifica que "VALOR_PASIVOS" no sea igual a 1
)

## Campo TOT_ACTIVOB

Regla 119 El campo debe ser numérico


In [ ]:
# Regla para validar que "TOT_ACTIVOB" sea numérico
TOT_ACTIVOB_Regla_SerNumerico = ~col('TOT_ACTIVOB').rlike('^-?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?$')

Regla 120 No deben estar en 0 (cero), nulo, 1 (uno)


In [ ]:
# Regla para validar que "TOT_ACTIVOBB" no sea 0, nulo, 1 o vacío
TOT_ACTIVOB_Regla_NoCeroNuloUnoVacio = (
    col('TOT_ACTIVOB').isNull() |                # Verifica que "TOT_ACTIVOBBBB" no sea nulo
    (col('TOT_ACTIVOB') == '') |                   # Verifica que "TOT_ACTIVOB" no esté vacío
    (col('TOT_ACTIVOB') == '0') |                    # Verifica que "TOT_ACTIVOB" no sea igual a 0
    (col('TOT_ACTIVOB') == '1')                      # Verifica que "TOT_ACTIVOB" no sea igual a 1
)

Regla 121 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún activo


In [ ]:
# Valor del Salario Mínimo Mensual Legal Vigente (SMMLV)
#SMMLV = 1160000  # Valor del SMMLV en pesos colombianos

# Calcular el 10% del SMMLV
limite_activos = 0.1  # 10% del SMMLV

# Regla para validar que "TOT_ACTIVOB" no esté por debajo del 10% del SMMLV
TOT_ACTIVOB_Regla_MayorQueCienMil = (
    col('TOT_ACTIVOB') <= limite_activos  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

Regl 122 No debe contener silabas o letras en minúscula o mayúscula combinadas en cualquier orden


In [ ]:
import re

# Regla para validar que "TOT_ACTIVOB" no contenga silabas o letras en minúscula o mayúscula combinadas
TOT_ACTIVOB_Regla_NoSilabas = (
    col('TOT_ACTIVOB').rlike(r'.*([aeiouAEIOU]{2,}|[a-zA-Z]{2,}).*') &  # Verifica la ausencia de combinaciones de letras o sílabas
    col('TOT_ACTIVOB').rlike(r'[^\d]')  # Verifica que el campo no contenga ningún carácter que no sea un dígito
)

## Campo REALIZA_OPERACIONES_ME

Regla 123 Campo no debe ser vacío o nulo


In [ ]:
# Regla para validar que "REALIZA_OPERACIONES_ME" no sea nulo ni vacío
REALIZA_OPERACIONES_ME_Regla_NoNuloNoVacio = (
    col('REALIZA_OPERACIONES_ME').isNull() |  # Verifica que "REALIZA_OPERACIONES_ME" no sea nulo
    (col('REALIZA_OPERACIONES_ME') == '')       # Verifica que "REALIZA_OPERACIONES_ME" no esté vacío
)

Regla 124 Debe ser Si o No solamente


In [ ]:
# Regla para validar que "REALIZA_OPERACIONES_ME" tenga solo los valores "Si" o "No"
REALIZA_OPERACIONES_ME_Regla_ValoresPermitidos = (
    ~col('REALIZA_OPERACIONES_ME').isin('S', 'N')  # Verifica si el valor está en la lista especificada
)

## Campo TIPO_PRODUCTO_ME

Regla 125 Obligatario si el campo REALIZA_OPERACIONES_ME es SI


In [ ]:
# Regla para validar que "TIPPRD1" no sea nulo ni vacío si "INDMONE" es "Si"
TIPO_PRODUCTO_ME_Regla_NoNuloNoVacioConIndicadorSi = (
    (col('REALIZA_OPERACIONES_ME') == 'S') &                    # Verifica que "INDMONE" sea "Si"
    (col('TIPO_PRODUCTO_ME').isNull() |  (col('TIPO_PRODUCTO_ME') == ''))              # Asegura que "TIPPRD1" no sea nulo
                                                                 # Asegura que "TIPPRD1" no esté vacío
)

Regla 126 Debe estar dentro de los parametros de las opciones a seleccionar


In [ ]:
TIPO_PRODUCTO_ME_Regla_ValoresPermitidos = (
    ~col('TIPO_PRODUCTO_ME').isin('0', '1', '2','3','4')  # Verifica si el valor está en la lista especificada
)

## Campo ENTIDAD_ME

Regla 127 Obligatorio si operaciones en el exterior


In [ ]:
# Regla para validar que "ENTIDAD_ME" no sea nulo ni vacío
ENTIDAD_ME_Regla_NoNuloNoVacio = (
    (col('REALIZA_OPERACIONES_ME') == 'S') & 
    (
        (col('ENTIDAD_ME').isNull() |          # Asegura que "ENTIDAD_ME" no sea nulo
        (col('ENTIDAD_ME') == '') ) 
                 )             # Asegura que "ENTIDAD_ME" no esté vacío
)   

Regla 128 Lista de monedas (listado)


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'ISO4217'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

ENTIDAD_ME_Regla_ListaISO = (
    ~col('ENTIDAD_ME').isin(valores_primera_columna)
)

Regla 129 "Si  el campo ""Cuentas_Exterior"" en los campos marcados con el número 1, para estos casos se debe validar que el campo “Moneda” esté diligenciado con texto (no números- ni caracteres especiales)"


In [ ]:
ENTIDAD_ME_Regla_NoTexto = (
    (col('Posee_Cuentas_Ext') == 'S') & (
        (col('ENTIDAD_ME').rlike('[^a-zA-Z]'))
    )
)

## Campo PAIS_ME

Regla 130 "Si  el campo ""Cuentas_Exterior"" en los campos marcados con el número 1, para estos casos se debe validar que el campo “Pais” esté diligenciado con texto (no números- ni caracteres especiales)"


In [ ]:
# Regla para validar que "PAIS_ME" contenga texto si "INDMONE" es "Si"
PAIS_ME_Regla_TextoConIndicadorSi =  (
    (col('Posee_Cuentas_Ext') == 'S') & (
        (col('PAIS_ME').rlike('[^a-zA-Z]'))
    )
)

Regla 131 Obligatorio si operaciones en el exterior


In [ ]:

PAIS_ME_Regla_NoNuloNoVacio = (
    (col('REALIZA_OPERACIONES_ME') == 'S') & 
    (
    (col('PAIS_ME').isNull() |             #
    (col('PAIS_ME') == '') )   
               )               #
)

Regla 132 Lista códigos países


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'Codigo Pais Dane'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

PAIS_ME_Regla_ListaPaises = (
    ~col('PAIS_ME').isin(valores_primera_columna)
)

## Campo COD_DEP_CONSTITUCIÓN

Regla 133 "Si el campo ""Cuentas_Exterior"" en los campos marcados con el número 1, para estos casos se debe validar que el campo “Ciudad” esté diligenciado (no vacio)"


In [ ]:
# Regla para validar que "COD_DEP_CONSTITUCIÓN" no sea nulo ni vacío
COD_DEP_CONSTITUCIÓN_Regla_NoNuloNoVacio = (
    (col('Posee_Cuentas_Ext') == 'S') & (
    col('COD_DEP_CONSTITUCIÓN').isNull() |          # Asegura que "COD_DEP_CONSTITUCIÓN" no sea nulo
    (col('COD_DEP_CONSTITUCIÓN') == '')        
                  )              # Asegura que "COD_DEP_CONSTITUCIÓN" no esté vacío
)

Regla 134 "Si  el campo ""Cuentas_Exterior"" en los campos marcados con el número 1, para estos casos se debe validar que el campo “Ciudad” esté diligenciado con texto (no números- ni caracteres especiales)"


In [ ]:
from pyspark.sql.functions import col, regexp_extract

# Regla para validar que "COD_DEP_CONSTITUCIÓN" contenga texto si "INDMONE" es "Si"
COD_DEP_CONSTITUCIÓN_Regla_TextoConIndicadorSi = (
    (col('Posee_Cuentas_Ext') == 'S') & (                                             # Verifica que "INDMONE" sea "Si"
    (col('COD_DEP_CONSTITUCIÓN').isNull() |                                      # Asegura que "COD_DEP_CONSTITUCIÓN" no sea nulo
    (col('COD_DEP_CONSTITUCIÓN') == '') |                                           # Asegura que "COD_DEP_CONSTITUCIÓN" no esté vacío
    (col('COD_DEP_CONSTITUCIÓN').rlike('[^a-zA-Z]')))
)
)

## Campo CUAL_CARGO

Regla 135 Solo es Obligatorio si es PEPS


In [ ]:
CUAL_CARGO_Regla_NoNuloVacio_PEPS_Juridicos = (
    (
    (col('FECINI_CARGO_PUBLICO') == 'S') & 
    (col('CUAL_CARGO').isNull() | (col('CUAL_CARGO') == ''))
    )|
    (
    (col('FECFIN_CARGO_PUBLICO') == 'S') & 
    (col('CUAL_CARGO').isNull() | (col('CUAL_CARGO') == ''))       
    )|
    (
    (col('DESEMP_CARGO_ACTUAL') == 'S') & 
    (col('CUAL_CARGO').isNull() | (col('CUAL_CARGO') == ''))      
    )
)

Regla 136 No se encuentre en lista de errados


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CARGO'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

CUAL_CARGO_Regla_NoListaErrados_Juridicos = (
    (
    (
    (col('FECINI_CARGO_PUBLICO') == 'S') & 
    ((col('CUAL_CARGO').isin(valores_primera_columna)))
    )|
    (
    (col('FECFIN_CARGO_PUBLICO') == 'S') & 
    ((col('CUAL_CARGO').isin(valores_primera_columna)))       
    )|
    (
    (col('DESEMP_CARGO_ACTUAL') == 'S') & 
    ((col('CUAL_CARGO').isin(valores_primera_columna)))      
    )
)
)

## Campo FECINI_CARGO_PUBLICO

Regla 137 No debe estar vacio/nulo/cero si tiene cargo publico


In [ ]:
FECINI_CARGO_PUBLICO_Regla_NoVacioPEPS_Juridicos =(
    (
    (col('CARGO_PUBLICO') == 'S') & 
    (col('FECINI_CARGO_PUBLICO').isNull() | (col('FECINI_CARGO_PUBLICO') == ''))
    )|
    (
    (col('DESEMP_CARGO_ACTUAL') == 'S') & 
    (col('FECINI_CARGO_PUBLICO').isNull() | (col('FECINI_CARGO_PUBLICO') == ''))
    )
)

Regla 138 Debe ser una fecha después del 01/01/1900


In [ ]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import lit

FECINI_CARGO_PUBLICO_Regla_FechaDespues1900_Juridicos = (
    (
    (col('CARGO_PUBLICO') == 'S') & 
    (~(regexp_extract(col('FECINI_CARGO_PUBLICO'), pattern, 0) == '') & ~(to_date(col('FECINI_CARGO_PUBLICO'), 'd/M/yyyy').cast('date') > lit('1900-01-01').cast('date'))  )
    )|
    (
    (col('DESEMP_CARGO_ACTUAL') == 'S') & 
    (~(regexp_extract(col('FECINI_CARGO_PUBLICO'), pattern, 0) == '') & ~(to_date(col('FECINI_CARGO_PUBLICO'), 'd/M/yyyy').cast('date') > lit('1900-01-01').cast('date'))   )
    )    
)

Regla 139 Estándar Format DD -MM-YYYY (ejemplo: 11-12-2024) 


In [ ]:
from pyspark.sql.functions import col, regexp_extract
# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FECINI_CARGO_PUBLICO_Regla_FormatoDiaMesAnio_Juridicos= (

        (
    (col('CARGO_PUBLICO') == 'S') & 
    ((regexp_extract(col('FECINI_CARGO_PUBLICO'), pattern, 0) == ''))
    )|
    (
    (col('DESEMP_CARGO_ACTUAL') == 'S') & 
    ((regexp_extract(col('FECINI_CARGO_PUBLICO'), pattern, 0) == '') )
    )
)

## Campo FECFIN_CARGO_PUBLICO

Regla 140 Debe ser una fecha después del 01/01/1900 en caso de que esté diligenciada


In [ ]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import lit


FECFIN_CARGO_PUBLICO_Regla_FechaDespues1900_Juridicos = (
    (col('CARGO_PUBLICO') == 'S') &           # Asegura que "FECINI_CARGO_PUBLICO" no sea nulo
    (col('DESEMP_CARGO_ACTUAL') == 'N') &
    ~(to_date(col('FECFIN_CARGO_PUBLICO'), 'd/M/yyyy').cast('date') > lit('1900-01-01').cast('date'))  
)

Regla 141 Estándar Format DD -MM-YYYY (ejemplo: 11-12-2024) 


In [ ]:
from pyspark.sql.functions import col, datediff, to_date, regexp_extract
# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FECFIN_CARGO_PUBLICO_Regla_FormatoDiaMesAnio_Juridicos= (
    
    col('FECFIN_CARGO_PUBLICO').isNotNull() &  # Asegura que el valor no sea nulo
    (col('FECFIN_CARGO_PUBLICO') != 0) &
    (regexp_extract(col('FECFIN_CARGO_PUBLICO'), pattern, 0) == '')  # Valida el formato usando una expresión regular
)

## Campo REP_LEGAL_ONG

Regla 142 Campo no vacio o nulo


In [ ]:
REP_LEGAL_ONG_Regla_NoVaciosNuloPEPS_Juridicos =(
    (
    (col('REP_LEGAL_ONG').isNull() | (col('REP_LEGAL_ONG') == ''))
    )|
    (
    (col('REP_LEGAL_ONG').isNull() | (col('REP_LEGAL_ONG') == ''))  
    )
)

Regla 143 Sí o No


In [ ]:
# Valores permitidos para el campo "REP_LEGAL_ONG"
valores_permitidos = ['S', 'N']

# Regla para validar que "REP_LEGAL_ONG" contenga solo los valores permitidos
REP_LEGAL_ONG_Regla_ValoresPermitidos_Juridicos = (
    ~col('REP_LEGAL_ONG').isin(valores_permitidos)  # Asegura que "REP_LEGAL_ONG" contenga solo los valores permitidos
)

## Campo CARGO_PUBLICO

Regla 144 No vacio o nulo si desempeña cargo publico o tiene fecha inicio cargo publico


In [ ]:
CARGO_PUBLICO_Regla_ObligatorioPEPS_Juriicos = (

    (
        (col('DESEMP_CARGO_ACTUAL') == 'S') & 
        (col('CARGO_PUBLICO').isNull() | (col('CARGO_PUBLICO') == '')) 
     ) |
        
    (   (col('FECINI_CARGO_PUBLICO') == 'S') & 
        (col('CARGO_PUBLICO').isNull() | (col('CARGO_PUBLICO') == '')) 
    ) 

)

Regla 145 Sí o No


In [ ]:
# Valores permitidos para el campo "CARGO_PUBLICO"
valores_permitidos = ['S', 'N']

CARGO_PUBLICO_Regla_ValoresPermitidos_Juridicos = (

    (
        (col('DESEMP_CARGO_ACTUAL') == 'S') & 
        (col('CARGO_PUBLICO').isNull() | (col('CARGO_PUBLICO') == '') | ~col('CARGO_PUBLICO').isin(valores_permitidos) ) 
     ) |
        
    (   (col('FECINI_CARGO_PUBLICO') == 'S') & 
        (col('CARGO_PUBLICO').isNull() | (col('CARGO_PUBLICO') == '') | ~col('CARGO_PUBLICO').isin(valores_permitidos)) 
    ) 

)

## Campo TIENE_VINCULO_PEPS

Regla 146 Campo vacio o nulo


In [ ]:
TIENE_VINCULO_PEPS_Regla_NoVaciosNuloPEPS_Juridico = (
    ((col('TIENE_VINCULO_PEPS').isNull() | (col('TIENE_VINCULO_PEPS') == '')) &
    (col('DESEMP_CARGO_ACTUAL') == 'S')) 
)

Regla 147 Sí o No


In [ ]:
# Valores permitidos para el campo "TIENE_VINCULO_PEPS"
valores_permitidos = ['S', 'N']

# Regla para validar que "TIENE_VINCULO_PEPS" contenga solo los valores permitidos
TIENE_VINCULO_PEPS_Regla_ValoresPermitidos_Juridico = (
    ~col('TIENE_VINCULO_PEPS').isin(valores_permitidos)  # Asegura que "TIENE_VINCULO_PEPS" contenga solo los valores permitidos
)

## SCORER

Regla 148 Alto o Bajo


In [ ]:
valores_permitidos = ['ALTO', 'BAJO', 'Alto', 'Bajo', 'alto', 'bajo']

# Regla para validar que "SCORER" contenga solo los valores permitidos
SCORER_Regla_ValoresPermitidos_Juridico = (
    ~col('SCORER').isin(valores_permitidos)  # Asegura que "SCORER" contenga solo los valores permitidos
)

## FEC_INGBCO

Regla 149 Alto o Bajo
 

In [ ]:
from pyspark.sql.functions import lit,col, to_date 
FEC_INGBCO_Regla_FechaDespues1900_Juridico = ~(                    # Asegura que "FEC_INGBCO" no sea nulo
    to_date(col('FEC_INGBCO'), 'd/M/yyyy').cast('date') > lit('1900-01-01').cast('date')                # Asegura que "FEC_INGBCO" sea después del 01/01/1900
)

Regla 150 Estándar Format DD -MM-YYYY (ejemplo: 11-12-2024) 


In [ ]:
from pyspark.sql.functions import col, datediff, to_date, regexp_extract
# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FEC_INGBCO_Regla_FormatoDiaMesAnio_Juridico= (
    col('FEC_INGBCO').isNotNull() &  # Asegura que el valor no sea nulo
    (regexp_extract(col('FEC_INGBCO'), pattern, 0) == '')  # Valida el formato usando una expresión regular
)

## Campo Fecuac

Regla 151 Estándar Format DD-MM-YYYY (ejemplo: 11-12-2024) 


In [ ]:
from pyspark.sql.functions import col, regexp_extract

# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FECUAC_Regla_FormatoAnioMesDia= (
    col('FECUAC').isNotNull() &  # Asegura que el valor no sea nulo
    (regexp_extract(col('FECUAC'), pattern, 0) == '')  # Valida el formato usando una expresión regular
)

Regla 152 Actualización de Datos según riesgo Sarlaft


In [ ]:
from pyspark.sql.functions import datediff, to_date, col


FECUAC_Regla_CoherenciaSegunRiesgo =(
    (col('FECUAC').isNull()) |  
    (col('FECUAC_EF').isNull()) | 
     
    (
        ((col("SCORER") == "ALTO") & (datediff(to_date(col('FechaCorte'), 'd/M/yyyy'), to_date(col('FECUAC'), 'd/M/yyyy')) > 365) |
         (col("SCORER") == "ALTO") & (datediff(to_date(col('FechaCorte'), 'd/M/yyyy'), to_date(col('FECUAC_EF'), 'd/M/yyyy')) > 365))
        | 
        ((col("SCORER") == "BAJO") & (datediff(to_date(col('FechaCorte'), 'd/M/yyyy'), to_date(col('FECUAC'), 'd/M/yyyy')) > (365 * 3)) |
         (col("SCORER") == "BAJO") & (datediff(to_date(col('FechaCorte'), 'd/M/yyyy'), to_date(col('FECUAC_EF'), 'd/M/yyyy')) > (365 * 3)))
    )
)

Regla 153 la fecha de vinculación-Ingreso del asociado no debe ser mayor a la fecha de actualización en el BUC  fecuact


In [ ]:
from pyspark.sql.functions import col, to_date

# Regla para validar que "FECUACT" sea menor que "FECUAC_EF"
FECUAC_Regla_MenorQueUltFecIng = (
    col('FECUAC').isNotNull() &                        # Asegura que el valor de "FECUACT" no sea nulo
    col('FEC_INGBCO').isNotNull() &                    # Asegura que el valor de "FECUAC_EF" no sea nulo
    (to_date(col('FECUAC'), 'd/M/yyyy').cast('date') < to_date(col('FEC_INGBCO'), 'd/M/yyyy'))               # Asegura que "FECUACT" sea menor que "FECUAC_EF"
)

Regla 154 mayor a19640303


In [ ]:
from pyspark.sql.functions import col, to_date, lit

# Regla para la columna "FECUACT"
FECUAC_Regla_MayorQue1964 = (
    (col('FECUAC').isNotNull() | (col('FECUAC') == '')) &  # Asegura que el valor no sea nulo
    (to_date(col('FECUAC'), 'd/M/yyyy') < lit('1964-03-03').cast('date'))  # Verifica si la fecha es mayor a '1964-03-03'
)

### Campo FECUAC_EF

Rega 155 mayor a19640303



In [ ]:
from pyspark.sql.functions import col, to_date, lit
FECUAC_EF_Regla_MayorQue1964 = (
    (col('FECUAC_EF').isNotNull() | (col('FECUAC_EF') == '')) &  # Asegura que el valor no sea nulo
    (to_date(col('FECUAC_EF'), 'd/M/yyyy') < lit('1964-03-03').cast('date'))  # Verifica si la fecha es mayor a '1964-03-03'
)

Regla 156 la fecha de ingreso del banco FEC_INGBCO no debe ser mayor a la fecha de actualización de datos financieros FECUAC_EF


In [ ]:
from pyspark.sql.functions import col, to_date

FECUAC_EF_Regla_NoMayorFechaFECUACT=  (
    to_date(col('FECUAC_EF'), 'd/M/yyyy').cast('date') < to_date(col('FEC_INGBCO'),'d/M/yyyy').cast('date')
)

Regla 157 Estándar Format DD -MM-YYYY (ejemplo: 11-12-2024) 


In [ ]:
from pyspark.sql.functions import col, datediff, to_date, regexp_extract
# Definir una regla para validar fechas en diferentes formatos
pattern = r'^\d{1,2}[/-]\d{1,2}[/-]\d{4}$'  # Acepta formatos con uno o dos dígitos para día y mes
FECUAC_EF_Regla_FormatoDiaMesAnio = (
    col('FECUAC_EF').isNotNull() &  # Asegura que el valor no sea nulo
    (regexp_extract(col('FECUAC_EF'), pattern, 0) == '')  # Valida el formato usando una expresión regular
)

## Campo NOM_PAI_REP

Regla 158 Campo no debe estar vacío o nulo


In [ ]:
NOM_PAI_REP_Regla_NoNuloNoVacio = (
    col('NOM_PAI_REP').isNull() |  # Asegura que el valor no sea nulo
    (col('NOM_PAI_REP') == '')       # Asegura que el valor no esté vacío
)

Regla 159 Validación de Tabla de países (hay una versión inicial en Golden Record); validar estándar ISO3166


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'Codigo Pais Dane'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

NOM_PAI_REP_Regla_ListaPaises = (
    ~col('NOM_PAI_REP').isin(valores_primera_columna)
)

## Campo COD_DEP_REP

Regla 160 Campo no debe ser vacío o nulo


In [ ]:
COD_DEP_REP_Regla_NoNuloNoVacio = (
    col('COD_DEP_REP').isNull() |  # Asegura que el valor no sea nulo
    (col('COD_DEP_REP') == '')       # Asegura que el valor no esté vacío
)

Regla 161 "El código de lugar de nacimiento o constitucion debe pertenecer al rango de opciones validas en parámetros // Códigos que esten dentro de la arhivo ""cliciudad"" de la Finclidat. (Códigos DANE)"


In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODDEP'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

COD_DEP_REP_Regla_ListaDepartamentos = (
    ~col('COD_DEP_REP').isin(valores_primera_columna)
)

Regla 162 Sólo Numérico


In [ ]:
COD_DEP_REP_Regla_SoloValoresNumericos = ~col('COD_DEP_REP').rlike('^[0-9]+$')

Regla 163 Diferente de 0 a 1


In [ ]:
COD_DEP_REP_Regla_DiferenteCeroUno = col('COD_DEP_REP').isin('0', '1')

## Campo ESTGESCOM

Regla 164 Campo no debe ser vacío o nulo


In [ ]:
ESTGESCOM_Regla_NoNuloNoVacio_jurididico = (
    col('ESTGESCOM').isNull() |  # Asegura que el valor no sea nulo
    (col('ESTGESCOM') == '')       # Asegura que el valor no esté vacío
)

Regla 165 El valor del ESTGESCOM debe ser ACTIVO o INACTIVO


In [ ]:
ESTGESCOM_Regla_EsActivoInactivo_Juridico = ~col('ESTGESCOM').isin('ACTIVO', 'INACTIVO')

Regla 166 No es Numérico


In [ ]:
ESTGESCOM_Regla_NoEsNumerico_Juridico = (col('ESTGESCOM').cast("float").isNotNull())

Regla 167 Diferente de 0 a 1


In [ ]:
ESTGESCOM_Regla_EsDiferenteCeroUno_Juridico = (col('ESTGESCOM') == 0) | (col('ESTGESCOM') == 1)

## Campo NOM_DEP_EXPEDICIÓN

Regla 168

In [ ]:
# Ahora carga el archivo CSV en PySpark
ruta_csv= "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')


nombre_primera_columna = 'CODDEP'


# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

NOM_DEP_EXPEDICIÓN_Regla_ListaDepartamentos = (
    ~col('NOM_DEP_EXPEDICIÓN').isin(valores_primera_columna)
)

## Campo TIPCLI

Regla 169 Campo no debe ser vacío o nulo


In [ ]:
TIPCLI_Regla_NoNuloNoVacio = (
    col('TIPCLI').isNull() |  # Asegura que el valor no sea nulo
    (col('TIPCLI') == '')       # Asegura que el valor no esté vacío
)

Regla 170 "El campo solo puede tener valores de (Persona Natural ""2"", Persona Jurídica ""1"")"


In [ ]:
TIPCLI_Regla_PerteneceLista = (
    ~col('TIPCLI').isin('2', '1' )
)

## Campo TIP_VINCLC

Regla 171 Campo no debe ser vacío o nulo


In [ ]:
TIP_VINCLC_Regla_NoNuloNoVacio = (
    col('TIP_VINCLC').isNull() |  # Asegura que el valor no sea nulo
    (col('TIP_VINCLC') == '')       # Asegura que el valor no esté vacío
)

Regla 172 El campo solo puede tener valores de (Solo Asociado, Solo Cliente, Mixto (Asociado-Cliente), No Cliente - No Asociado


In [ ]:
TIP_VINCLC_Regla_PerteneceLista = (
    ~col('TIP_VINCLC').isin('*No Cliente-No Asociado*', 'Mixto (Asociado/Cliente)', 'Solo Asociado', 'Solo Cliente')
)

## Campo REGIONAL


Regla 173 Campo no debe ser vacío o nulo


In [ ]:
REGIONAL_Regla_NoNuloNoVacio_Juridico = (
    col('REGIONAL').isNull() |  # Asegura que el valor no sea nulo
    (col('REGIONAL') == '')       # Asegura que el valor no esté vacío
)

Regla 174 "El campo solo puede tener valores de ( Bogota ""5"" Cali ""1"" Eje Cafetero ""2"" Medellin ""3"" Palmira ""4"" Caribe ""6"" Negocios Dirección Nacional ""9"")"


In [ ]:
REGIONAL_Regla_PerteneceLista_Juridico  = (
    ~col('REGIONAL').isin('1', '2', '3', '4', '5', '6', '9')
)

## Campo AGEHOMO

Regla 175 Campo no debe ser vacío o nulo


In [ ]:
AGEHOMO_Regla_NoNuloNoVacio_Juridico = (
    col('AGEHOMO').isNull() |  # Asegura que el valor no sea nulo
    (col('AGEHOMO') == '')       # Asegura que el valor no esté vacío
)

## Campo DESEMP_CARGO_ACTUAL

Regla 176 Obligatorio si tiene Cargo Publico o fecha de inicio de cargo publico


In [ ]:
from pyspark.sql.functions import col, to_date, lit
DESEMP_CARGO_ACTUAL_Regla_ObligatorioPEPS_Juridico = (

    (
        (col('CARGO_PUBLICO') == 'S') & 
        (col('DESEMP_CARGO_ACTUAL').isNull() | (col('DESEMP_CARGO_ACTUAL') == '')) 
     ) |
        
    (   (col('FECINI_CARGO_PUBLICO') == 'S') & 
        (col('DESEMP_CARGO_ACTUAL').isNull() | (col('DESEMP_CARGO_ACTUAL') == '')) 
    ) 
)

## Campo COD_DEP_CONSTITUCIÓN

Regla 177 Campo no debe ser vacío o nulo


In [ ]:
COD_DEP_CONSTITUCIÓN_Regla_NoVaciosNulos = (
    col('COD_DEP_CONSTITUCIÓN').isNull() |  # Asegura que el valor no sea nulo
    (col('COD_DEP_CONSTITUCIÓN') == '')    # Asegura que el valor no esté vacío
)

Regla 178 Códigos departamento parametrizados DANE


In [ ]:
# Cargar el archivo CSV en PySpark
ruta_csv = "/home/jovyan/code/Bancoomeva_Oficial/ListaExternaValoresComparar.csv"
df_csv = spark.read.csv(ruta_csv, header=True, sep=';')

# Seleccionar la primera columna del DataFrame por su nombre y obtener una lista de sus valores
nombre_primera_columna = 'CODDEP'
valores_primera_columna = [row[0] for row in df_csv.select(nombre_primera_columna).collect()]

# Definir el filtro para el campo COD_DEP_CONSTITUCIÓN
COD_DEP_CONSTITUCIÓN_Regla_ListaDepartamentos = (
    ~col('COD_DEP_CONSTITUCIÓN').isin(valores_primera_columna)
)

## Campo NUMERO_SOCIOS

Regla 179 Sólo Numérico


In [ ]:
NUMERO_SOCIOS_Regla_Numerico = ~col('NUMERO_SOCIOS').cast('bigint').rlike('^[0-9]+$')

Regla 180 Debe ser mayor a CERO, si el valor de TIPO_EMPRESA está clasificado como SI en accionistas

In [ ]:
from pyspark.sql.functions import col

# Lista de valores para el campo "TIPO_EMPRESA" que deben ser considerados
valores_tipo_empresa_permitidos = [
    "Comandita Simple",
    "Comandita por Acciones",
    "EMPRESAR SECTOR DEL GOBIERNO",
    "ESTABLECIMIENTO PUBLICO",
    "LIMITADA",
    "Ninguno/No Definido",
    "OTRO",
    "SOCIEDAD ANONIMA",
    "Sociedad Anonima Simplificada",
    "SOCIEDAD DE ECONOMIA MIXTA",
    "SOCIEDAD PRIVADA EXTRANJERA",
    "SOCIEDAD PRIVADA NACIONAL",
    "Unipersonal"
]

# Crear el filtro
NUMERO_SOCIOS_Regla_ValoresTipoEmpresaAccionista = (
    (col("NUMERO_SOCIOS").cast("bigint").isNull()) |  # Verifica si el valor no es un número
    (col("NUMERO_SOCIOS").cast("bigint") == 0)       # Verifica si el valor es cero
) & (
    col("TIPO_EMPRESA").isin(valores_tipo_empresa_permitidos)  # Verifica si el tipo de empresa está en la lista permitida
)


## Campo IDENTIFICACION_SOCIO1

Regla 181 Campo no debe ser vacío o nulo, si el valor de TIPO_EMPRESA esta clasificado como SI en accionistas


In [ ]:
from pyspark.sql.functions import col

# Lista de valores para el campo "TIPO_EMPRESA" que deben ser considerados
valores_tipo_empresa_permitidos = [
    "Comandita Simple",
    "Comandita por Acciones",
    "EMPRESAR SECTOR DEL GOBIERNO",
    "ESTABLECIMIENTO PUBLICO",
    "LIMITADA",
    "Ninguno/No Definido",
    "OTRO",
    "SOCIEDAD ANONIMA",
    "Sociedad Anonima Simplificada",
    "SOCIEDAD DE ECONOMIA MIXTA",
    "SOCIEDAD PRIVADA EXTRANJERA",
    "SOCIEDAD PRIVADA NACIONAL",
    "Unipersonal"
]

# Crear el filtro
IDENTIFICACION_SOCIO1_Regla_NoVacioNulo = (
    (col("TIPO_EMPRESA").isin(valores_tipo_empresa_permitidos)) &  # Verifica si el tipo de empresa está en la lista permitida
    ((col("IDENTIFICACION_SOCIO1").isNull()) | (col("IDENTIFICACION_SOCIO1") == ""))  # Verifica si la identificación del socio 1 es nula o vacía
)


Regla 182 Solo debe tener los códigos de tipo de documento TI, CC, CE, Pasaporte, NIT


In [ ]:
# Lista de tipos de documento válidos
tipos_documento_validos = ["4", "1", "2", "5", "3"]

# Crear el filtro
IDENTIFICACION_SOCIO1_Regla_TiposDeDocumentoValidos = (
    ~col("TIPDOC").isin(tipos_documento_validos)
)

Regla 183 Valida que el número de documento y tipo de documento estén completos en los dos campos, si uno de los dos campos hace falta se etiqueta como NO VALIDO.


In [ ]:
# Crear el filtro
# IDENTIFICACION_SOCIO1_Regla_campos_incompletos = (
#    (col("IDENTIFICACION_SOCIO1").isNull() | (col("IDENTIFICACION_SOCIO1") == "")) |  # Verifica si el campo IDENTIFICACION_SOCIO1 es nulo o vacío
#    (col("TIPDOC").isNull() | (col("TIPDOC") == ""))  # Verifica si el campo TIPDOC es nulo o vacío
# )

Regla 184 Si el tipo de documento es Cedula de ciudadanía se valida que el número de documento sea de 6, 7 u 8 dígitos y puede comenzar con cualquier valor numérico


In [ ]:
from pyspark.sql.functions import col

# Regla para validar el campo IDENTIFICACION_SOCIO1 si TIPDOC es igual a "1" pero el número de documento no tiene 6, 7, 8 u 10 dígitos
IDENTIFICACION_SOCIO1_Regla_Tipdoc_1_NoTieneDigitosAdecuados = (
    (col('TIPDOC') == "1") &  # Verifica si TIPDOC es igual a "1"
    (
        (~col('IDENTIFICACION_SOCIO1').rlike('^\d{6}$')) |  # Verifica que la IDENTIFICACION_SOCIO1 no tenga 6 dígitos
        (~col('IDENTIFICACION_SOCIO1').rlike('^\d{7}$')) |  # Verifica que la IDENTIFICACION_SOCIO1 no tenga 7 dígitos
        (~col('IDENTIFICACION_SOCIO1').rlike('^\d{8}$')) |  # Verifica que la IDENTIFICACION_SOCIO1 no tenga 8 dígitos
        (~col('IDENTIFICACION_SOCIO1').rlike('^\d{10}$'))    # Verifica que la IDENTIFICACION_SOCIO1 no tenga 10 dígitos
    )
)

Regla 185 Si el tipo de documento es Cedula de ciudadanía se valida que el número de documento sea de 10 dígitos siempre y cuando éste comience por el digito 1.


In [ ]:
# Filtro para seleccionar los registros con TIPDOC "1", el primer valor del campo IDENTIFICACION_SOCIO1 es '1', pero no tiene 6, 7, 8 u 10 dígitos
IDENTIFICACION_SOCIO1_Regla_Tipdoc_1_PrimerDigito1_NoTieneDigitosAdecuados = (
    (col('TIPDOC') == '1') &  # Verifica si TIPDOC es igual a "1"
    col('IDENTIFICACION_SOCIO1').rlike('^1') &  # Verifica si el primer valor del IDENTIFICACION_SOCIO1 es '1'
    (~col('IDENTIFICACION_SOCIO1').rlike('^\d{6}$') &  # Verifica que el IDENTIFICACION_SOCIO1 no tenga 6 dígitos
     ~col('IDENTIFICACION_SOCIO1').rlike('^\d{7}$') &  # Verifica que el IDENTIFICACION_SOCIO1 no tenga 7 dígitos
     ~col('IDENTIFICACION_SOCIO1').rlike('^\d{8}$') &  # Verifica que el IDENTIFICACION_SOCIO1 no tenga 8 dígitos
     ~col('IDENTIFICACION_SOCIO1').rlike('^\d{10}$')  # Verifica que el IDENTIFICACION_SOCIO1 no tenga 10 dígitos
    )
)

Regla 186 Si Tipo de Identificacion es 'CE', el dato debe contener (6) caracteres alfanúmericos.


In [ ]:
# Filtro para seleccionar los registros con TIPDOC "2" y el campo IDENTIFICACION_SOCIO1 no contiene 6 caracteres alfanuméricos
IDENTIFICACION_SOCIO1_Regla_Tipdoc_2_No6Caracteres = (
    (col('TIPDOC') == '2') &  # Verifica si TIPDOC es igual a "2"
    ~(length('IDENTIFICACION_SOCIO1') == 6)  # Verifica si la longitud del campo IDENTIFICACION_SOCIO1 no es 6
)

Regla 187 Si el tipo de documento es cedula de Extranjería, puede tener la letra E al inicio, seguido de un digito de 1 a 9 y continuar con dígitos de 0 a 9 con una longitud total de 5, 6 o 7 dígitos, para un total máximo de 9 caracteres


In [ ]:
from pyspark.sql.functions import length, col

# Filtro para seleccionar los registros con TIPDOC "2" y el campo IDENTIFICACION_SOCIO1 no cumple con el patrón o longitud especificada
IDENTIFICACION_SOCIO1_Regla_Tipdoc_CedulaExtranjeria_NoPatron = (
    (col('TIPDOC') == '2') &  # Verifica si TIPDOC es igual a "Cédula de Extranjería"
    ~col('IDENTIFICACION_SOCIO1').rlike('^[A-Za-z0-9][1-9][0-9]{4,6}$') |  # Verifica el patrón del IDENTIFICACION_SOCIO1 modificado
    ~(length(col('IDENTIFICACION_SOCIO1')) >= 7) | ~(length(col('IDENTIFICACION_SOCIO1')) <= 9)  # Regla para longitud total
)

Regla 188 Si el tipo de documento es Pasaporte el número de documento puede tener o no dos letras al inicio y continuar con dígitos de 0 a 9 con una longitud total de 6, 7, 8, 9 o 10 dígitos


In [ ]:
# Filtro para seleccionar los registros con TIPDOC "5" y el campo IDENTIFICACION_SOCIO1 no cumple con la longitud especificada
IDENTIFICACION_SOCIO1_Regla_Longitud_PA_NoCumpleCondicion = (
    (col('TIPDOC') == '5') &  # Verifica si TIPDOC es igual a "PA"
    ~(col('IDENTIFICACION_SOCIO1').rlike('^[0-9A-Za-z]{6,16}$'))  # IDENTIFICACION_SOCIO1 contiene entre 6 y 16 caracteres alfanuméricos
)


Regla 189 Si el tipo de documento es NIT el número de documento debe comenzar por 8 o 9, seguido de 8 dígitos que pueden estar entre 0,9 y terminar con el digito de verificación.


In [ ]:
from pyspark.sql.functions import col

# Filtro para registros donde TIPDOC es igual a "NIT" pero el campo IDENTIFICACION_SOCIO1 no sigue el patrón específico
IDENTIFICACION_SOCIO1_Regla_Tipdoc_NIT_NoCumplePatron = (
    (col('TIPDOC') == '3') &  # Verifica si TIPDOC es igual a "NIT"
    ~(col('IDENTIFICACION_SOCIO1').rlike('^[89]\\d{8}'))  # Verificación del patrón específico del IDENTIFICACION_SOCIO1 para NIT
)


Regla 190 Campo debe ser único en la combinación tipo de identificación + nitcli


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

def filtro_registros_duplicados_identificacion_tipdoc(df_spark: DataFrame) -> 'Column':
    # Cálculo de registros con duplicados en la combinación de 'IDENTIFICACION_SOCIO1' y 'TIPDOC'
    identificacion_Regla_unicidadCombinacionIdentificacionMasTipo = (
        df_spark
        .groupBy('IDENTIFICACION_SOCIO1', 'TIPDOC','FechaCorte')
        .agg(F.count('*').alias('conteo'))
        .filter("conteo > 1")
    )

    # Obtener una lista de 'IDENTIFICACION_SOCIO1' que tienen duplicados en la combinación de 'IDENTIFICACION_SOCIO1' y 'TIPDOC'
    identificaciones_con_duplicados = identificacion_Regla_unicidadCombinacionIdentificacionMasTipo.select('IDENTIFICACION_SOCIO1').collect()

    # Convertir la lista de 'IDENTIFICACION_SOCIO1' en una lista de valores para filtrar
    lista_identificaciones_con_duplicados = [row['IDENTIFICACION_SOCIO1'] for row in identificaciones_con_duplicados]

    # Aplicar el filtro para seleccionar aquellos registros cuya 'IDENTIFICACION_SOCIO1' está en la lista de duplicados
    filtro_unicidad = col('IDENTIFICACION_SOCIO1').isin(lista_identificaciones_con_duplicados)

    return filtro_unicidad

# Llamar a la función para obtener el filtro
IDENTIFICACION_SOCIO1_Regla_unicidadCombinacionIdentificacionMasTipo = filtro_registros_duplicados_identificacion_tipdoc(df_spark)


## Campo TIPO_EMPRESA 

Regla 191 El valor del campo debe corresponder al catálogo de datos TIPO_EMPRESAS

In [ ]:
from pyspark.sql.functions import col

# Lista de valores permitidos para el campo TIPO_EMPRESA
valores_permitidos_tipo_empresa = [
    "ASOCIACION FONDO EMPLEADOS O GREMIAL",
    "Comandita Simple",
    "Comandito por Acciones",
    "De Hecho",
    "EMPRESAR SECTOR DEL GOBIERNO",
    "ENTIDAD COOPERATIVA",
    "ENTIDAD DEL ESTADO",
    "ENTIDAD SIN ANIMO DE LUCRO",
    "ESTABLECIMIENTO PUBLICO",
    "LIMITADA",
    "Ninguno/No Definido",
    "OTRO",
    "SOCIEDAD ANONIMA",
    "Sociedad Anonima Simplificada",
    "SOCIEDAD DE ECONOMIA MIXTA",
    "SOCIEDAD PRIVADA EXTRANJERA",
    "SOCIEDAD PRIVADA NACIONAL",
    "Union Temporal Consorcio",
    "Unipersonal"
]

# Crear el filtro para seleccionar los registros que no están en la lista de valores permitidos para TIPO_EMPRESA
TIPO_EMPRESA_Regla_valoresPermitidos = ~col("TIPO_EMPRESA").isin(valores_permitidos_tipo_empresa)


## Campo NOMBRE_SOCIO1

Regla 386 Primer nombre y primer apellido deben ser obligatorios. Segundo nombre y segundo apellido pueden ser nulos.

In [ ]:
NOMBRE_SOCIO1_Regla_PrimerNombre_PrimerApellido = size(split(col('NOMBRE_SOCIO1'), ' ')) < 2

Regla 387  Campo no debe contener números (el dato contenga caracteres alfabeticos)

In [ ]:
NOMBRE_SOCIO1_Regla_NoNumeros = col('NOMBRE_SOCIO1').rlike(r'\d')

Regla 388 Campo no debe contener caracteres especiales (a excepción de la Ñ)

In [ ]:
NOMBRE_SOCIO1_Regla_NoCaracteresEspeciales = col('NOMBRE_SOCIO1').rlike(r'[^a-zA-ZñÑ\s]')

Regla 389 Validación de espacios entre segmento de caracteres(palabras)

In [ ]:
NOMBRE_SOCIO1_Regla_AlMenosUnEspacio = (
    ~col('NOMBRE_SOCIO1').rlike(r'\s')  # Verifica si hay al menos un espacio en el campo
)

Regla 390 Que no exista solo un punto (.), coma (,), asteriscos (*)


In [ ]:
NOMBRE_SOCIO1_Regla_NoPuntosComasAsteriscos = col('NOMBRE_SOCIO1').rlike(r'[.,*]')

Regla 391 No debe contener sucesion del mismo carácter mas de 3 veces (XXXX, AAAAAA,etc.)

In [ ]:
NOMBRE_SOCIO1_Regla_NoSecuenciaRepetida = col('NOMBRE_SOCIO1').rlike(r'(.)\1{3,}')

Regla 391  El campo no debe contener  una sola letra minúscula o mayúscula


In [ ]:
NOMBRE_SOCIO1_Regla_NoUnaSolaLetra = ~col('NOMBRE_SOCIO1').rlike(r'^(?![a-zA-Z]$).*$')

Regla 392 El campo no debe tener con silabas compuestas por 2 letras, es decir, dos letras combinadas o repetidas de cualquier forma en minúscula o mayúscula


In [ ]:
NOMBRE_SOCIO1_Regla_NoSilabasDosLetras = col('NOMBRE_SOCIO1').rlike(r'\b([a-zA-Z])\1\b')

## Campo TOT_PTRMNOB

Regla No Debe contener ceros

In [ ]:
TOT_PTRMNOB_Regla_Cero = (
    col('TOT_GASTOSB').isin(0)  # Verifica que "TOT_GASTOSB" sea 0
)

Regla 393 El campo debe ser numérico


In [ ]:
TOT_PTRMNOB_Regla_SerNumerico =  (
    ~col('TOT_PTRMNOB').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$')
)

Regla 394 No deben estar en 0 (cero), nulo, 1 (uno)


In [ ]:
TOT_PTRMNOB_Regla_NoUnoNuloVacio = (
    col('TOT_PTRMNOB').isNull() |               # Verifica que "TOT_PTRMNOB" no sea nulo
    (col('TOT_PTRMNOB') == '') |                  # Verifica que "TOT_PTRMNOB" no esté vacío                   # Verifica que "TOT_PTRMNOB" no sea igual a 0
    (col('TOT_PTRMNOB') == 1)                      # Verifica que "TOT_PTRMNOB" no sea igual a 1
)

Regla 395 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún egreso


In [ ]:
# Valor del Salario Mínimo Mensual Legal Vigente (SMMLV)
#SMMLV = 1160000  # Por ejemplo, el SMMLV en Colombia es 1.160.000 pesos

# Calcular el 10% del SMMLV
limite_egreso = 0.1  # 10% del SMMLV

# Regla para validar que "TOT_PTRMNOB" no esté por debajo del 10% del SMMLV
TOT_PTRMNOB_Regla_NoMenorQueCienMil = (
    col('TOT_PTRMNOB') <= limite_egreso  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

## Campo CAP_SOCIALB

Regla 396 El campo debe ser numérico

In [ ]:
CAP_SOCIALB_Regla_SerNumerico =  (
    ~col('CAP_SOCIALB').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$')
)

Regla No cero

In [ ]:
CAP_SOCIALB_Regla_Cero = (
    col('CAP_SOCIALB').isin(0)  # Verifica que "TOT_GASTOSB" sea 0
)

Regla 397 No deben estar en 0 (cero), nulo, 1 (uno)

In [ ]:
CAP_SOCIALB_Regla_NoUnoNuloVacio = (
    col('CAP_SOCIALB').isNull() |               # Verifica que "CAP_SOCIALB" no sea nulo
    (col('CAP_SOCIALB') == '') |                  # Verifica que "CAP_SOCIALB" no esté vacío                   # Verifica que "CAP_SOCIALB" no sea igual a 0
    (col('CAP_SOCIALB') == 1)                      # Verifica que "CAP_SOCIALB" no sea igual a 1
)

Regla 398 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún egreso

In [ ]:
# Valor del Salario Mínimo Mensual Legal Vigente (SMMLV)
#SMMLV = 1160000  # Por ejemplo, el SMMLV en Colombia es 1.160.000 pesos

# Calcular el 10% del SMMLV
limite_egreso = 0.1  # 10% del SMMLV

# Regla para validar que "CAP_SOCIALB" no esté por debajo del 10% del SMMLV
CAP_SOCIALB_Regla_NoMenorQueLimiteCienMil = (
    col('CAP_SOCIALB') <= limite_egreso  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

## Campo INGRESOS_MENSUALES

Regla 399 El campo debe ser numérico

In [ ]:
INGRESOS_MENSUALES_Regla_SerNumerico =  (
    ~col('INGRESOS_MENSUALES').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$')
)

Regla No ceros

In [ ]:
INGRESOS_MENSUALES_Regla_Cero = (
    col('INGRESOS_MENSUALES').isin(0)  # Verifica que "TOT_GASTOSB" sea 0
)

Regla 400 No deben estar en 0 (cero), nulo, 1 (uno)

In [ ]:
# Regla para validar que "INGRESOS_MENSUALES" no sea 0, 1, nulo o vacío
INGRESOS_MENSUALES_Regla_NoUnoNuloVacio = (
    col('INGRESOS_MENSUALES').isNull() |               # Verifica que "INGRESOS_MENSUALES" no sea nulo
    (col('INGRESOS_MENSUALES') == '') |                  # Verifica que "INGRESOS_MENSUALES" no esté vacío                   # Verifica que "INGRESOS_MENSUALES" no sea igual a 0
    (col('INGRESOS_MENSUALES') == 1)                      # Verifica que "INGRESOS_MENSUALES" no sea igual a 1
)

Regla 401 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún egreso

In [ ]:
# Valor del Salario Mínimo Mensual Legal Vigente (SMMLV)
#SMMLV = 1160000  # Por ejemplo, el SMMLV en Colombia es 1.160.000 pesos

# Calcular el 10% del SMMLV
limite_egreso =  0.1  # 10% del SMMLV

# Regla para validar que "INGRESOS_MENSUALES" no esté por debajo del 10% del SMMLV
INGRESOS_MENSUALES_Regla_NoMenorQueCienMil = (
    col('INGRESOS_MENSUALES') <= limite_egreso  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

## Campo EGRESOS_MENSUALES

Regla 402 El campo debe ser numérico


In [ ]:
EGRESOS_MENSUALES_Regla_SerNumerico =  (
    ~col('EGRESOS_MENSUALES').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$')
)

Regla No Ceros

In [ ]:
EGRESOS_MENSUALES_Regla_Cero = (
    col('EGRESOS_MENSUALES').isin(0)  # Verifica que "TOT_GASTOSB" sea 0
)

Regla 403 No deben estar en 0 (cero), nulo, 1 (uno)

In [ ]:
EGRESOS_MENSUALES_Regla_NoUnoNuloVacio = (
    col('EGRESOS_MENSUALES').isNull() |               # Verifica que "EGRESOS_MENSUALES" no sea nulo
    (col('EGRESOS_MENSUALES') == '') |                  # Verifica que "EGRESOS_MENSUALES" no esté vacío                   # Verifica que "EGRESOS_MENSUALES" no sea igual a 0
    (col('EGRESOS_MENSUALES') == 1)                      # Verifica que "EGRESOS_MENSUALES" no sea igual a 1
)

Regla 404 No deben estar  < 10% SMMLV, es decir que no cuentan con ningún egreso

In [ ]:
#SMMLV = 1160000  # Por ejemplo, el SMMLV en Colombia es 1.160.000 pesos

# Calcular el 10% del SMMLV
limite_egreso =  0.1  # 10% del SMMLV

# Regla para validar que "EGRESOS_MENSUALES" no esté por debajo del 10% del SMMLV
EGRESOS_MENSUALES_Regla_NoMenorQueCienMil = (
    col('EGRESOS_MENSUALES') <= limite_egreso  # Verifica que el valor sea mayor o igual al 10% del SMMLV
)

## Campo MONTO_ME

Regla 405 El campo debe ser numérico si MARCA SI en MONEDA EXTRANJERA

In [ ]:


MONTO_ME_Regla_numericoSiMonedaExtranjera = (

    (col('REALIZA_OPERACIONES_ME') == 'S') & 
                                        
    (~col('MONTO_ME').rlike('^[0-9]+([,][0-9]+)?([Ee][+-]?[0-9]+)?$'))  
          
)

Regla 406 No deben estar en 0 (cero), nulo, 1 (uno) si MARCA SI en MONEDA EXTRANJERA

In [ ]:
MONTO_ME_Regla_numericoCeroUnoNulo = (
    (col('REALIZA_OPERACIONES_ME') == 'S') &
    (
        (col('MONTO_ME').isNull()) | 
        (col('MONTO_ME') == 0) | 
        (col('MONTO_ME') == 1) |
        (col('MONTO_ME') == '')
    )
)

## Campo Posee_Cuentas_Ext

Regla 407 Debe permitir solo el valor N o S

In [ ]:
Posee_Cuentas_Ext_Regla_ValoresPermitidos = (
    ~col('Posee_Cuentas_Ext').isin('S', 'N')  # Verifica si el valor está en la lista especificada
)

Regla 408 El campo no puede ser vacio

In [ ]:
Posee_Cuentas_Ext_Regla_NoNuloNoVacio = (
    col('Posee_Cuentas_Ext').isNull() |  # Verifica que "Posee_Cuentas_Ext" no sea nulo
    (col('Posee_Cuentas_Ext') == '')       # Verifica que "Posee_Cuentas_Ext" no esté vacío
)

# Jerarquia De Analisis QA

In [ ]:
## Diccionario con la Jerarquia Campo:Dimension:Regla

from pyspark.sql import functions as F

reglas_campos = {

    'REP_LEGAL': {
        'Completitud': [REP_LEGAL_Regla_PrimerNombre_PrimerApellido],
        'Validez': [REP_LEGAL_Regla_NoNumeros, REP_LEGAL_Regla_NoCaracteresEspeciales, REP_LEGAL_Regla_AlMenosUnEspacio],
        'Exactitud':[REP_LEGAL_Regla_NoPuntosComasAsteriscos,REP_LEGAL_Regla_NoSecuenciaRepetida,REP_LEGAL_Regla_NoUnaSolaLetra,
                     REP_LEGAL_Regla_NoSilabasDosLetras] 
        # Otras dimensiones y sus respectivos filtros para el campo 'NOMCLI'
    }, 
    

    'FECHA_DE_CONSTITUCION': {
        'Completitud': [FECHA_DE_CONSTITUCION_Regla_NoVacio],
        'Exactitud': [FECHA_DE_CONSTITUCION_Regla_NoFuturo,FECHA_DE_CONSTITUCION_Regla_MenorIgualQue1900, FECHA_DE_CONSTITUCION_Regla_Mayor120Anios, 
                      FECHA_DE_CONSTITUCION_Regla_NoMayorFechaIngreso],
        'Validez': [FECHA_DE_CONSTITUCION_Regla_FormatoDiaMesAnio],             
    }, 

    'CODIGO_LUGAR_DE_CONSTITUCION': { 
        'Completitud': [CODIGO_LUGAR_DE_CONSTITUCION_Regla_NoVacioONulo],
        'Consistencia': [CODIGO_LUGAR_DE_CONSTITUCION_Regla_ValorPerteneceListaDane],
        'Validez': [CODIGO_LUGAR_DE_CONSTITUCION_Regla_SoloNumerico, CODIGO_LUGAR_DE_CONSTITUCION_Regla_IgualA01],
    },

    'TIPDOC': { 
        'Completitud': [TIPDOC_Regla_NuloOVacio_j]
    },


    'NIT': { 
        'Completitud': [NIT_Regla_NuloOVacio_j],
        'Consistencia': [NIT_Regla_NoCompletosTIPDOC_j,NIT_Regla_Tipdoc_CC_NoTieneDigitosAdecuados_j, 
                         NIT_Regla_Tipdoc_CC_PrimerDigito1_NoTiene10Digitos_j, NIT_Regla_Tipdoc_TI_PatronEspecifico_siInicia1_j,
                          NIT_Regla_Tipdoc_RegistroCivil_NoCumple_j, 
                         NIT_Regla_Longitud_CE_NoCumpleCondicion_j,NIT_Regla_Tipdoc_CedulaExtranjeria_NoPatron_j,                        
                         NIT_Regla_Tipdoc_Pasaporte_NoCumplePatron_j,NIT_Regla_Tipdoc_NIT_NoCumplePatron_j, 
                         NIT_Regla_Tipdoc_PP_NoCumplePatron_j, NIT_Regla_Tipdoc_7_NoCumpleCondicion_j,
                         NIT_Regla_Longitud_PA_NoCumpleCondicion_j],
        'Unicidad': [NIT_Regla_unicidadCombinacionNitMasTipo_j],
        'Validez': [NIT_Regla_Tipdoc_CC_LongitudExcedida_j]
    },

    'FECEXPREP': {
        'Completitud': [FECEXPREP_Regla_NoVacio],    
        'Validez': [FECEXPREP_Regla_NoEsTexto],
        'Exactitud': [FECEXPREP_Regla_DiferenciaMayorIgual120,FECEXPREP_Regla_FechaAnterior1900]
    }, 

    'CIUDAD_EXPEDICION': { 
        'Completitud': [CIUDAD_EXPEDICION_Regla_NoVacioONulo], 
        'Validez': [CIUDAD_EXPEDICION_Regla_No0o1,CIUDAD_EXPEDICION_Regla_SoloNumerico],
        'Consistencia': [CIUDAD_EXPEDICION_Regla_ValorPerteneceListaDane]
    }, 

    'DIRECCION_COMERCIO': {
        'Completitud': [DIRECCION_COMERCIO_Regla_NoVacioONulo],
        'Validez': [DIRECCION_COMERCIO_Regla_NoCaracteresEspeciales, DIRECCION_COMERCIO_Regla_ValorListaErrados],
        'Exactitud': [DIRECCION_COMERCIO_Regla_NoSoloNumeros,DIRECCION_COMERCIO_Regla_NoSoloUnaPalabra,DIRECCION_COMERCIO_Regla_ContieneNoAplica],
        'Consistencia': [DIRECCION_COMERCIO_Regla_LongitudMinima]
    }, 

    'CODIGO_CIUDAD_DE_SEDE_PRINCIPAL': { 
        'Completitud': [CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_NoVacioNiNulo, CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_NoVacioNiNulo_SinDireccionBarrioDiligenciado],
        'Validez': [CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_EsNumerico, CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_ValorListaDane],
        'Exactitud': [CODIGO_CIUDAD_DE_SEDE_PRINCIPAL_Regla_ValorCodigoCero]
    },


    'TEL_REP': {
        'Completitud': [TEL_REP_Regla_NoNuloOVacio],
        'Validez': [TEL_REP_Regla_Numerico,TEL_REP_Regla_Longitud10],
        'Exactitud': [TEL_REP_Regla_NoUnaSolaLetra,TEL_REP_Regla_NoNoAplica, TEL_REP_Regla_NoTexto],
        'Consistencia': [TEL_REP_Regla_ValorListaErrados]
    }, 

    'TEL_COMERC': {
        'Completitud': [TEL_COMERC_Regla_VacioNuloSinTEL_REP, TEL_COMERC_Regla_vaciosConDirecciones],
        'Validez': [TEL_COMERC_Regla_SoloNumeros,TEL_COMERC_Regla_Longitud,TEL_COMERC_Regla_NoMenorQue999999,
                    TEL_COMERC_Regla_MayorQue1000000,TEL_COMERC_Regla_Patron],
        'Exactitud': [TEL_COMERC_Regla_NoLetraSola,TEL_COMERC_Regla_NoSilabasRepetidas,TEL_COMERC_Regla_NoContieneNoAplica]
    }, 


     'NOM_PAIS_CONSTITUCIÓN': {
        'Completitud': [NOM_PAIS_CONSTITUCIÓN_Regla_NoVacioNulo],
        'Validez': [NOM_PAIS_CONSTITUCIÓN_Regla_ValorListaDane]
    }, 

    'NOMBRE': {  
        'Validez': [NOMBRE_Regla_NoNumeros,NOMBRE_Regla_NoCaracteresEspeciales,NOMBRE_Regla_NoSoloPuntoComaAsterisco,NOMBRE_Regla_SinPuntosComas],
        'Exactitud': [NOMBRE_Regla_NoSecuenciaRepetida,NOMBRE_Regla_NoSoloNumerosLetras],
        'Consistencia': [NOMBRE_Regla_NoListaErrados]
    }, 

    'Dir_Representante': {
        'Validez': [Dir_Representante_Regla_CaracteresEspeciales],
        'Exactitud': [Dir_Representante_Regla_NoSoloNumerico,Dir_Representante_Regla_NoSilabasEspecíficas, Dir_Representante_Regla_NoSoloUnaPalabra,
                         Dir_Representante_Regla_ContieneNoAplica, Dir_Representante_Regla_NoSoloNumerosNiUnaLetra],
        'Consistencia': [Dir_Representante_Regla_LongitudMayorA7]
    }, 

    'CIU_REP': {
        'Validez': [CIU_REP_Regla_ValorListaDane],
    }, 

    'CIIU': {
        'Completitud': [CIIU_Regla_NoCeroONuloOVacio],
        'Consistencia': [CIIU_Regla_ValorListaCIIU]
    }, 

    'TOT_VENTASB': {
        'Completitud': [TOT_VENTASB_Regla_Cero,TOT_VENTASB_Regla_NoUnoNuloOVacio, TOT_VENTASB_Regla_NoVacioConActividadesEspecificas, 
                        TOT_VENTASB_Regla_NumeroConActividadesEspecificas],
        'Validez': [TOT_VENTASB_Regla_EsNumerico,],
        'Consistencia': [TOT_VENTASB_Regla_NoMenorQueCienMil]
    }, 

    'OTR_INGRSOB': {
        'Validez': [OTR_INGRSOB_Regla_EsNumerico],
        'Consistencia': [OTR_INGRSOB_Regla_CumpleCondiciones],
        'Exactitud': [OTR_INGRSOB_Regla_NoTexto],
    }, 

    'DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES': {
        'Completitud': [DETALLE_DE_OTROS_INGRESOS_NO_OPERACIONALES_Regla_TextoSinDigitos],
    }, 

    'TOT_GASTOSB': {
        'Validez': [TOT_GASTOSB_Regla_SerNumerico],
        'Completitud': [TOT_GASTOSB_Regla_Cero,TOT_GASTOSB_Regla_NoUnoNuloVacio],
        'Consistencia': [TOT_GASTOSB_Regla_NoMenorQueCienMil]
    }, 

    'TOT_PASIVOB': {
        'Consistencia': [TOT_PASIVOB_Regla_MayorQueCero],
        'Completitud': [TOT_PASIVOB_Regla_NoUnoNuloVacio]
    }, 

    'TOT_ACTIVOB': {
        'Validez': [TOT_ACTIVOB_Regla_SerNumerico],
        'Completitud': [TOT_ACTIVOB_Regla_NoCeroNuloUnoVacio],
        'Consistencia': [TOT_ACTIVOB_Regla_MayorQueCienMil],
        'Exactitud': [TOT_ACTIVOB_Regla_NoSilabas]
    }, 

    'REALIZA_OPERACIONES_ME': { 
        'Completitud': [REALIZA_OPERACIONES_ME_Regla_NoNuloNoVacio],
        'Validez': [REALIZA_OPERACIONES_ME_Regla_ValoresPermitidos],
    }, 

    'TIPO_PRODUCTO_ME': {
        'Completitud': [TIPO_PRODUCTO_ME_Regla_NoNuloNoVacioConIndicadorSi],
        'Validez': [TIPO_PRODUCTO_ME_Regla_ValoresPermitidos]
    }, 

    'ENTIDAD_ME': {
        'Completitud': [ENTIDAD_ME_Regla_NoNuloNoVacio],
        'Validez': [ENTIDAD_ME_Regla_ListaISO, ENTIDAD_ME_Regla_NoTexto]
    }, 

    'PAIS_ME': {
        'Completitud': [PAIS_ME_Regla_TextoConIndicadorSi, PAIS_ME_Regla_NoNuloNoVacio],
        'Validez': [PAIS_ME_Regla_ListaPaises]
    }, 

    'COD_DEP_CONSTITUCIÓN': {
        'Completitud': [COD_DEP_CONSTITUCIÓN_Regla_NoNuloNoVacio, COD_DEP_CONSTITUCIÓN_Regla_NoNuloNoVacio],
        'Validez': [COD_DEP_CONSTITUCIÓN_Regla_TextoConIndicadorSi, COD_DEP_CONSTITUCIÓN_Regla_ListaDepartamentos]
    }, 
 

    'CUAL_CARGO': {
        'Completitud': [CUAL_CARGO_Regla_NoNuloVacio_PEPS_Juridicos],
        'Validez': [CUAL_CARGO_Regla_NoListaErrados_Juridicos]
    },

    'FECINI_CARGO_PUBLICO': {
        'Completitud': [FECINI_CARGO_PUBLICO_Regla_NoVacioPEPS_Juridicos],
        'Consistencia': [FECINI_CARGO_PUBLICO_Regla_FechaDespues1900_Juridicos],
        'Validez': [FECINI_CARGO_PUBLICO_Regla_FormatoDiaMesAnio_Juridicos]
    }, 

    'FECFIN_CARGO_PUBLICO': {
        
        'Consistencia': [FECFIN_CARGO_PUBLICO_Regla_FechaDespues1900_Juridicos],
        'Validez': [FECFIN_CARGO_PUBLICO_Regla_FormatoDiaMesAnio_Juridicos]
    }, 

    'REP_LEGAL_ONG': {
        'Completitud': [REP_LEGAL_ONG_Regla_NoVaciosNuloPEPS_Juridicos],
        'Validez': [REP_LEGAL_ONG_Regla_ValoresPermitidos_Juridicos]
    }, 

    'CARGO_PUBLICO': {
        'Completitud': [CARGO_PUBLICO_Regla_ObligatorioPEPS_Juriicos],
        'Validez': [CARGO_PUBLICO_Regla_ValoresPermitidos_Juridicos]
    }, 

    'TIENE_VINCULO_PEPS': { 
        'Completitud': [TIENE_VINCULO_PEPS_Regla_NoVaciosNuloPEPS_Juridico],
        'Validez': [TIENE_VINCULO_PEPS_Regla_ValoresPermitidos_Juridico]
    }, 


    'SCORER': {
        'Completitud': [SCORER_Regla_ValoresPermitidos_Juridico]
    }, 


    'FEC_INGBCO': {
        'Consistencia': [FEC_INGBCO_Regla_FechaDespues1900_Juridico],        
        'Validez': [FEC_INGBCO_Regla_FormatoDiaMesAnio_Juridico]
    }, 

    'FECUAC': {
        'Validez': [FECUAC_Regla_FormatoAnioMesDia],
        'Actualizacion': [FECUAC_Regla_CoherenciaSegunRiesgo],
        'Consistencia': [FECUAC_Regla_MenorQueUltFecIng, FECUAC_Regla_MayorQue1964]
    }, 

    'FECUAC_EF': {
        'Consistencia': [FECUAC_EF_Regla_MayorQue1964, FECUAC_EF_Regla_NoMayorFechaFECUACT],
        'Validez': [FECUAC_EF_Regla_FormatoDiaMesAnio]
    },

    'NOM_PAI_REP': {
        'Completitud': [NOM_PAI_REP_Regla_NoNuloNoVacio],
        'Validez': [NOM_PAI_REP_Regla_ListaPaises]
    },

    'COD_DEP_REP': {
        'Completitud': [COD_DEP_REP_Regla_NoNuloNoVacio],
        'Consistencia': [COD_DEP_REP_Regla_ListaDepartamentos],
        'Validez': [COD_DEP_REP_Regla_SoloValoresNumericos, COD_DEP_REP_Regla_DiferenteCeroUno]
    },

    'ESTGESCOM': {
        'Completitud': [ESTGESCOM_Regla_NoNuloNoVacio_jurididico],
        'Consistencia': [ESTGESCOM_Regla_EsActivoInactivo_Juridico],
        'Validez': [ESTGESCOM_Regla_NoEsNumerico_Juridico, ESTGESCOM_Regla_EsDiferenteCeroUno_Juridico]
    },

    'NOM_DEP_EXPEDICIÓN': {
        'Validez': [NOM_DEP_EXPEDICIÓN_Regla_ListaDepartamentos]
    },

    'TIPCLI': {
        'Completitud': [TIPCLI_Regla_NoNuloNoVacio],
        'Exactitud': [TIPCLI_Regla_PerteneceLista,]
    },

    'TIP_VINCLC': {
        'Completitud': [TIP_VINCLC_Regla_NoNuloNoVacio],
        'Exactitud': [TIP_VINCLC_Regla_PerteneceLista]
    },

    'REGIONAL': {
        'Completitud': [REGIONAL_Regla_NoNuloNoVacio_Juridico],
        'Exactitud': [REGIONAL_Regla_PerteneceLista_Juridico]
    },

    'AGEHOMO': {
        'Completitud': [AGEHOMO_Regla_NoNuloNoVacio_Juridico]
    },

    'DESEMP_CARGO_ACTUAL': {
        'Completitud': [DESEMP_CARGO_ACTUAL_Regla_ObligatorioPEPS_Juridico]
    },

    'NUMERO_SOCIOS': {
    'Consistencia': [NUMERO_SOCIOS_Regla_Numerico],
    'Validez': [NUMERO_SOCIOS_Regla_ValoresTipoEmpresaAccionista] 
    },

    'IDENTIFICACION_SOCIO1': {
    'Completitud': [IDENTIFICACION_SOCIO1_Regla_NoVacioNulo],    
    'Consistencia': [IDENTIFICACION_SOCIO1_Regla_TiposDeDocumentoValidos],    
    'Validez': [IDENTIFICACION_SOCIO1_Regla_Tipdoc_1_NoTieneDigitosAdecuados,IDENTIFICACION_SOCIO1_Regla_Tipdoc_1_PrimerDigito1_NoTieneDigitosAdecuados,
                IDENTIFICACION_SOCIO1_Regla_Tipdoc_2_No6Caracteres, IDENTIFICACION_SOCIO1_Regla_Tipdoc_CedulaExtranjeria_NoPatron, IDENTIFICACION_SOCIO1_Regla_Longitud_PA_NoCumpleCondicion,
                IDENTIFICACION_SOCIO1_Regla_Tipdoc_NIT_NoCumplePatron,IDENTIFICACION_SOCIO1_Regla_unicidadCombinacionIdentificacionMasTipo]         
    },

    'TIPO_EMPRESA': {
    'Consistencia': [TIPO_EMPRESA_Regla_valoresPermitidos]
    },

    'NOMBRE_SOCIO1': {
    'Completitud': [NOMBRE_SOCIO1_Regla_PrimerNombre_PrimerApellido],
    'Validez':[NOMBRE_SOCIO1_Regla_NoNumeros, NOMBRE_SOCIO1_Regla_NoCaracteresEspeciales,NOMBRE_SOCIO1_Regla_AlMenosUnEspacio] , 
    'Exactitud': [NOMBRE_SOCIO1_Regla_NoPuntosComasAsteriscos, NOMBRE_SOCIO1_Regla_NoSecuenciaRepetida, NOMBRE_SOCIO1_Regla_NoUnaSolaLetra, NOMBRE_SOCIO1_Regla_NoSilabasDosLetras]  
    },

    'TOT_PTRMNOB': {
    'Validez': [TOT_PTRMNOB_Regla_SerNumerico],
    'Completitud': [TOT_PTRMNOB_Regla_Cero,TOT_PTRMNOB_Regla_NoUnoNuloVacio],
    'Consistencia': [TOT_PTRMNOB_Regla_NoMenorQueCienMil]
    },

    'CAP_SOCIALB': {
    'Validez': [CAP_SOCIALB_Regla_SerNumerico],
    'Completitud': [CAP_SOCIALB_Regla_Cero,CAP_SOCIALB_Regla_NoUnoNuloVacio],
    'Consistencia': [CAP_SOCIALB_Regla_NoMenorQueLimiteCienMil]
    },

    'INGRESOS_MENSUALES': {
    'Validez': [INGRESOS_MENSUALES_Regla_SerNumerico],
    'Completitud': [INGRESOS_MENSUALES_Regla_Cero,INGRESOS_MENSUALES_Regla_NoUnoNuloVacio],
    'Consistencia': [INGRESOS_MENSUALES_Regla_NoMenorQueCienMil]
    },   

    'EGRESOS_MENSUALES': {
    'Validez': [EGRESOS_MENSUALES_Regla_SerNumerico],
    'Completitud': [EGRESOS_MENSUALES_Regla_Cero,EGRESOS_MENSUALES_Regla_NoUnoNuloVacio],
    'Consistencia': [EGRESOS_MENSUALES_Regla_NoMenorQueCienMil]
    }, 

    'MONTO_ME': {
    'Validez': [MONTO_ME_Regla_numericoSiMonedaExtranjera],
    'Completitud': [MONTO_ME_Regla_numericoCeroUnoNulo]    
    },

    'Posee_Cuentas_Ext': {
    'Validez': [Posee_Cuentas_Ext_Regla_ValoresPermitidos],
    'Completitud': [Posee_Cuentas_Ext_Regla_NoNuloNoVacio]    
    },    
    
}

# Ejecución automatizada de Calidad

In [ ]:
from pyspark.sql import functions as F

def get_variable_name(variable):
    for name in globals():
        if id(globals()[name]) == id(variable):
            return name
    for name in locals():
        if id(locals()[name]) == id(variable):
            return name
    return None

from pyspark.sql import functions as F

def generar_dataframe_desde_campo(campo,dataframe):
    # Obtener las reglas y filtros para el campo especificado
    reglas_campo = reglas_campos.get(campo, {})

    # Obtener el DataFrame original solo con las columnas del campo y 'NIT'
    df_campo = dataframe.select(campo, 'NIT', 'FechaCorte')

    # Lista para almacenar los DataFrames filtrados
    dataframes_concatenados = []

    # Iterar sobre las dimensiones y filtros del campo especificado
    for dimension, filtros in reglas_campo.items():
        # Iterar sobre los filtros
        for filtro in filtros:
            # Aplicar el filtro al DataFrame del campo
            df_filtrado = df_campo.filter(filtro)
            
            # Obtener el nombre del filtro como cadena
            nombre_filtro = get_variable_name(filtro)
            
            # Agregar la columna 'Regla' con el nombre de la regla aplicada
            regla_column = F.lit(f"{nombre_filtro}")
            df_filtrado = df_filtrado.withColumn('Regla', regla_column)
            
            # Cambiar el nombre de la columna del campo a 'Registro'
            df_filtrado = df_filtrado.withColumnRenamed(campo, 'Registro')
            
            # Agregar la columna 'Dimension' con el nombre de la dimensión
            df_filtrado = df_filtrado.withColumn('Dimension', F.lit(dimension))
            
            # Agregar la columna 'Campo' con el nombre del campo
            df_filtrado = df_filtrado.withColumn('Campo', F.lit(campo))
            
            # Agregar el DataFrame filtrado a la lista
            dataframes_concatenados.append(df_filtrado)

    # Concatenar todos los DataFrames en la lista
    df_concatenado = dataframes_concatenados[0] if dataframes_concatenados else None
    for i in range(1, len(dataframes_concatenados)):
        df_concatenado = df_concatenado.union(dataframes_concatenados[i])

    return df_concatenado


Se hace el bucle

In [ ]:
# Obtener la lista de campos desde el diccionario de reglas_campos
lista_campos_reglas = list(reglas_campos.keys())

# DataFrame vacío para almacenar los resultados concatenados
resultado_concatenado = None

# Iterar sobre cada campo en la lista
for campo in lista_campos_reglas:
    try:
        # Llamar al método 'generar_dataframe_desde_campo' con el campo actual
        df_actual = generar_dataframe_desde_campo(campo,df_spark)
        
        # Verificar si es el primer DataFrame, si es así, asignar directamente el resultado
        if resultado_concatenado is None:
            resultado_concatenado = df_actual
        else:
            # Concatenar el DataFrame actual con el DataFrame acumulado
            resultado_concatenado = resultado_concatenado.union(df_actual)
    
    except Exception as e:
        # Manejar la excepción (puedes imprimir un mensaje de error o simplemente omitirlo)
        print(f"Error al procesar el campo '{campo}': {str(e)}")
        
        continue  # Ir al siguiente campo si ocurre una excepción

# Poblamiento 

## Tabla CampoActivo 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, row_number
from pyspark.sql.window import Window


lista_campos_reglas = list(reglas_campos.keys())
elementos_a_eliminar = ('BARRIO')

# Eliminar elementos de la lista
lista_campos_reglas = [campo for campo in lista_campos_reglas if campo not in elementos_a_eliminar]
df_pob_campoActivo = df_spark.select(lista_campos_reglas)

# Obtener información sobre nombre, tipo de dato y longitud de cada columna en df_pob_campoActivo
column_info = [(col.name, str(col.dataType), len(col.name)) for col in df_pob_campoActivo.schema]

# Crear DataFrame en PySpark con la información obtenida
df_campoActivo = spark.createDataFrame(column_info, ["Campo", "tipo", "longitud"])

# Agregar las columnas adicionales
df_campoActivo = df_campoActivo.withColumn("descripcion", lit("descripción del campo"))
df_campoActivo = df_campoActivo.withColumn("id_activo", lit(2))

# Agregar una columna 'id_campoActivo' con valores incrementales secuenciales
windowSpec = Window.orderBy(lit(1))
df_campoActivo = df_campoActivo.withColumn("id_campoActivo", row_number().over(windowSpec))

In [ ]:
import pandas as pd

# Convertir el DataFrame de PySpark a DataFrame de Pandas
df_pandas = df_campoActivo.toPandas()

# Especificar la ruta donde se guardará el archivo Excel
ruta_excel = "informacionCamposPanda.xlsx"  # Reemplaza con la ruta deseada y el nombre del archivo

# Guardar el DataFrame de Pandas en un archivo Excel
df_pandas.to_excel(ruta_excel, index=False)


## Tabla Regla

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, row_number
from pyspark.sql.window import Window

# Lista para almacenar todas las reglas
lista_reglas = []
lista_reglasTecnicas= []

# Iterar sobre el diccionario para obtener las reglas
for campo, dimensiones in reglas_campos.items():
    for dimension, reglas in dimensiones.items():
        lista_reglas.extend([get_variable_name(regla) for regla in reglas])
        lista_reglasTecnicas.extend([f"{campo}.{str(regla)}" for regla in reglas])

# Lista para almacenar los datos para el DataFrame
data = []

# Iterar sobre el diccionario para obtener las reglas
for campo, dimensiones in reglas_campos.items():
    for dimension, reglas in dimensiones.items():
        for regla in reglas:
            regla_tecnica= lista_reglasTecnicas.pop(0)
            regla_nombre = lista_reglas.pop(0)
            data.append({'Regla': regla_nombre, 'nombre Campo': campo, 'dimension': dimension, 'regla Tecnica': regla_tecnica})

# Crear el DataFrame df_regla con los datos
df_regla = spark.createDataFrame(data)

# Asignar un ID único y secuencial a cada regla como identificador primario
windowSpec = Window.orderBy(lit(1))
df_regla = df_regla.withColumn("id_regla", row_number().over(windowSpec) + 203)  # Comenzando desde "204"

# Agregar una columna 'id_activo' con el valor constante 1
df_regla = df_regla.withColumn("id_activo", lit(2))


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, row_number
from pyspark.sql.window import Window

# Ruta del archivo CSV
ruta_csv = "/home/jovyan/code/Bancoomeva_Oficial/ReglasNegocioVersion2.csv"

# Leer el archivo CSV en un DataFrame de PySpark
df_csv = spark.read.option("header", "true").option("delimiter", ";").csv(ruta_csv)

# Selecciona la columna DescripcionRegla
df_reglasNegocio = df_csv.select('DescripcionRegla')

# Añadir columna 'ID' con identificadores únicos al DataFrame
windowSpec = Window.orderBy(lit(1))
df_reglasNegocio = df_reglasNegocio.withColumn("id_regla", row_number().over(windowSpec) + 203)  # Comenzando desde "204"

# Asegúrate de tener la columna 'id_regla' como tipo Integer
df_reglasNegocio = df_reglasNegocio.withColumn("id_regla", col("id_regla").cast("int"))

# Muestra el DataFrame resultante
df_reglasNegocio.show()

# Unir los DataFrames por la columna 'id_regla'
df_unido = df_reglasNegocio.join(df_regla, "id_regla")


+--------------------+--------+
|    DescripcionRegla|id_regla|
+--------------------+--------+
|Primer nombre y p...|     204|
|Campo no debe con...|     205|
|Campo no debe con...|     206|
|Validación de esp...|     207|
|Que no exista sol...|     208|
|No debe contener ...|     209|
| El campo no debe...|     210|
|El campo no debe ...|     211|
|Campo no debe ser...|     212|
|No debe ser mayor...|     213|
|Debe ser mayor a ...|     214|
|(FECHA NACIMIENTO...|     215|
|No debe ser mayor...|     216|
|Estándar ISO8601F...|     217|
|Campo no debe ser...|     218|
|"""El código de l...|     219|
|       Sólo Numérico|     220|
|  Diferente de 0 a 1|     221|
|Campo no debe ser...|     222|
|Campo no debe ser...|     223|
+--------------------+--------+
only showing top 20 rows



In [ ]:
#cargar_dataframe_a_tabla(df_unido, "Calidad.ReglaPj")

## Tabla RegistrosNoConformes

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import to_date

# Agregar la columna "ID_analisisQA" con valor constante "1" al DataFrame "resultado_concatenado"
RegistroNoConforme = resultado_concatenado.withColumn("ID_analisisQA", lit(1))


# Crear una ventana de partición para ordenar los registros y generar un ID único
windowSpec = Window.orderBy(lit(1))
RegistroNoConforme = RegistroNoConforme.withColumn("ID_Registro", row_number().over(windowSpec))

df_reordenado = RegistroNoConforme.select('ID_Registro', 'NIT', 'Registro', 'Regla', 'Dimension', 'Campo','ID_analisisQA', 'FechaCorte')

df_resultadoReg = df_reordenado.join(df_regla.select("Regla", "id_regla"), on="Regla", how="left")
df_resultadoReg = df_resultadoReg.join(df_campoActivo.select("Campo", "id_campoActivo"), on="Campo", how="left")

In [ ]:
df_resultadoReg = df_resultadoReg.withColumn("id_activo", lit(2))

In [ ]:


num_filas = df_resultadoReg.count()

# Obtener el número de columnas
num_columnas = len(df_resultadoReg.columns)

print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

# Mostrar el esquema del DataFrame obtenido
df_resultadoReg.printSchema()

El DataFrame tiene 390909 filas y 11 columnas.
root
 |-- Campo: string (nullable = false)
 |-- Regla: string (nullable = false)
 |-- ID_Registro: integer (nullable = false)
 |-- NIT: long (nullable = true)
 |-- Registro: string (nullable = true)
 |-- Dimension: string (nullable = false)
 |-- ID_analisisQA: integer (nullable = false)
 |-- FechaCorte: string (nullable = true)
 |-- id_regla: integer (nullable = true)
 |-- id_campoActivo: integer (nullable = true)
 |-- id_activo: integer (nullable = false)



In [ ]:
cargar_dataframe_a_tabla(df_resultadoReg,"Calidad.RegCortesRemPJ")

# PRUEBAS

In [ ]:
#from pyspark.sql.functions import col


#[TEL_REP_Regla_NoNuloOVacio],
#[TEL_REP_Regla_Numerico,TEL_REP_Regla_Longitud10],
#[TEL_REP_Regla_NoUnaSolaLetra,TEL_REP_Regla_NoNoAplica, TEL_REP_Regla_NoTexto],
#[TEL_REP_Regla_ValorListaErrados 

#df_campo = df_spark.select('NIT','TEL_REP')
#df_campo = df_remediado.select('TEL_RESIDE')

# Aplicar el filtro y mostrar los resultados
#df1 = df_campo.filter(TEL_REP_Regla_Longitud10)
#df1.show(truncate=True)

#num_filas = df1.count()

# Obtener el número de columnas
#num_columnas = len(df1.columns)

#print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")